In [1]:
# -*- coding: utf-8 -*-
import sys
import string

try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

import struct
import numpy as np



def myint(x):
    # two bytes length
    # print(type(x), type(x[0]), type(ord(x[0])))

    # ans = ord(x[0])+(ord(x[1])<<8)

    ans = struct.unpack('h',x)[0]
    return ans

def mylong(x):
    # four bytes length
    if sys.version_info[0] > 2:
        ans = x[0] + (x[1]<<8) + (x[2]<<16) + (x[3]<<32)
    else:
        ans = ord(x[0]) + (ord(x[1])<<8) + (ord(x[2])<<16) + (ord(x[3])<<32)
    return ans

def myfloat(x):
    ans = struct.unpack('f',x)[0]
    # ans = ans[0]
    return ans

def mydouble(x):
    ans = struct.unpack('d',x)[0]
    # ans = ans[0]
    return ans

def mybyte(x):
    # will return an error if x is more than a byte length
    # TODO need to write error catch routine
    ans = ord(x)
    return ans

def mypass(x):
    return x

def myRGB(x):
    # three bytes with RGB values 
    # TODO need to write error catch routine
    R = x[0] if sys.version_info[0] > 2 else ord(x[0])
    G = x[1] if sys.version_info[0] > 2 else ord(x[1])
    B = x[2] if sys.version_info[0] > 2 else ord(x[2])

    ans = '#{:02x}{:02x}{:02x}'.format(R,G,B)
    return ans

def terminateStrOnZero (str):
    """Iterate through string and terminate on first zero
    """
    idx=0
    while idx < len(str) and str[idx] != '\00':
        idx += 1
    return str[:idx]

################################################################
class PTWFrameInfo:
  """Class to store the ptw file header information.
  """

  def __init__(self):

    """
        Initializes an object with several empty or zeroed properties. 

        These properties are used to store information from various sections of a 
        file's header and frame data, including name, signature, format, date and 
        time of file save, camera details, lens details, filter details, aperture 
        details, billet details, temperature details, pixel details, housing 
        temperature, serial number, threshold levels, unit details, lock details, 
        isoterm details, palette details, coordinates details, and other various 
        properties. 

        Args:
            self: The instance of the class.

        Returns:
            None
        """

    self.FileName = ''
    self.h_Signature = '' #[0:5]
    self.h_format = 'unknown'
    self.h_unit = ''
    self.h_Version = '' #[5:10]
    self.h_EofAsciiCode = 0 #[10]
    self.h_MainHeaderSize = 0 #[11:15]
    self.h_FrameHeaderSize = 0 #[15:19]
    self.h_SizeOfOneFrameAndHeader = 0 #[19:23]
    self.h_SizeOfOneFrame = 0 #[23:27]
    self.h_NumberOfFieldInFile = 0 #[27:31]
    self.h_CurrentFieldNumber = 0 #[31:35]

    #self.h_FileSaveDate = '' #[35:39]
    self.h_FileSaveYear = 0
    self.h_FileSaveMonth = 0
    self.h_FileSaveDay = 0

    #self.h_FileSaveTime = '' #[39:43]
    self.h_FileSaveHour = 0
    self.h_FileSaveMinute = 0
    self.h_FileSaveSecond = 0
    self.h_FileSaveCent = 0

    self.h_Millieme = 0 #[43:44]
    self.h_CameraName = '' #[44:64]
    self.h_LensName = '' #[64:84]
    self.h_FilterName = '' #[84:104]
    self.h_ApertureName = '' #[104:124]
    self.h_IRUSBilletSpeed = 0 #[124:128] # IRUS
    self.h_IRUSBilletDiameter = 0 #myfloat(headerinfo[128:132]) # IRUS
    self.h_IRUSBilletShape = 0 #myint(headerinfo[132:134]) #IRUS
    self.h_Emissivity = 0 #myfloat(headerinfo[141:145])
    self.h_Ambiant = 0 #myfloat(headerinfo[145:149])
    self.h_Distance = 0 #myfloat(headerinfo[149:153])
    self.h_IRUSInductorCoil = 0 #ord(everthing[153:154]) # IRUS
    self.h_IRUSInductorPower = 0 #mylong(headerinfo[154:158]) # IRUS
    self.h_IRUSInductorVoltage = 0 #myint(headerinfo[158:160]) # IRUS
    self.h_IRUSInductorFrequency = 0 #mylong(headerinfo[160:164]) # IRUS
    self.h_IRUSSynchronization = 0 #ord(headerinfo[169:170]) # IRUS
    self.h_AtmTransmission = 0 #myfloat(headerinfo[170:174])
    self.h_ExtinctionCoeficient = 0 #myfloat(headerinfo[174:178])
    self.h_Object = 0 #myint(headerinfo[178:180])
    self.h_Optic = 0 #myint(headerinfo[180:182])
    self.h_Atmo = 0 #myint(headerinfo[182:184])
    self.h_AtmosphereTemp = 0 #myfloat(headerinfo[184:188])
    self.h_CutOnWavelength = 0 #myfloat(headerinfo[188:192])
    self.h_CutOffWavelength = 0 #myfloat(headerinfo[192:196])
    self.h_PixelSize = 0 #myfloat(headerinfo[196:200])
    self.h_PixelPitch = 0 #myfloat(headerinfo[200:204])
    self.h_DetectorApperture = 0 #myfloat(headerinfo[204:208])
    self.h_OpticsFocalLength = 0 #myfloat(headerinfo[208:212])
    self.h_HousingTemperature1 = 0 #myfloat(headerinfo[212:216])
    self.h_HousingTemperature2 = 0 #myfloat(headerinfo[216:220])
    self.h_CameraSerialNumber = '' #headerinfo[220:231])
    self.h_MinimumLevelThreshold = 0 #myint(headerinfo[245:247])
    self.h_MaximumLevelThreshold = 0 #myint(headerinfo[247:249])
    self.h_EchelleSpecial = 0 #myint(headerinfo[277:279])
    self.h_EchelleUnit = 0 #headerinfo[279:289]
    self.h_EchelleValue = 0 #(headerinfo[289:357]) # 16 float values
    self.h_Units = ''
    self.h_Lockin = 0
    self.h_LockinGain = 0 #myfloat(headerinfo[357:361])
    self.h_LockinOffset = 0 #myfloat(headerinfo[361:365])
    self.h_HorizontalZoom = 0 #myfloat(headerinfo[365:369])
    self.h_VerticalZoom = 0 #myfloat(headerinfo[369:373])
    self.h_PixelsPerLine = 0 #myint(headerinfo[377:379])
    self.h_LinesPerField = 0 #myint(headerinfo[379:381])
    self.h_Rows = 0
    self.h_Cols = 0

    self.h_framepointer=1
    self.h_firstframe=1
    self.h_cliprect=[0,0,1,1]
    self.h_lastframe=0
    self.h_FrameSize = 0

    self.h_ADDynamic = 0 #myint(headerinfo[381:383])
    self.h_SATIRTemporalFrameDepth = 0 #myint(headerinfo[383:385]) # SATIR
    self.h_SATIRLocationLongitude = 0 #myfloat(headerinfo[385:389]) # SATIR
    self.h_SATIRLocationLatitude = 0 #myfloat(headerinfo[389:393]) # SATIR South is negative
    self.h_SATIRLocationAltitude = 0 #myfloat(headerinfo[393:397]) # SATIR
    self.h_ExternalSynch = 0 #ord(headerinfo[397]) # 1=External 0 = Internal
    self.h_CEDIPAquisitionPeriod = 0 #myfloat(headerinfo[403:407]) # CEDIP seconds
    self.h_CEDIPIntegrationTime = 0 #myfloat(headerinfo[407:411]) # CEDIP seconds
    self.h_WOLFSubwindowCapability = 0 #myint(headerinfo[411:413]) # WOLF
    self.h_ORIONIntegrationTime = 0 #myfloat(headerinfo[413:437]) # ORION (6 values)
    self.h_ORIONFilterNames = '' #headerinfo[437:557]) # ORION 6 fields of 20 chars each
    self.h_NucTable = 0 #myint(headerinfo[557:559])
    self.h_Reserve6 = '' #headerinfo[559:563]
    self.h_Comment = '' #headerinfo[563:1563]
    self.h_CalibrationFileName = '' #headerinfo[1563:1663]
    self.h_ToolsFileName = '' #headerinfo[1663:1919]
    self.h_PaletteIndexValid = 0 #ord(headerinfo[1919:1920])
    self.h_PaletteIndexCurrent = 0 #myint(1920:1922])
    self.h_PaletteToggle = 0 #ord(headerinfo[1922:1923])
    self.h_PaletteAGC = 0 #ord(headerinfo[1923:1924])
    self.h_UnitIndexValid = 0 #ord(headerinfo[1924:1925])
    self.h_CurrentUnitIndex = 0 #myint(headerinfo[1925:1927])
    self.h_ZoomPosition = 0 #(headerinfo[1927:1935]) # unknown format POINT
    self.h_KeyFrameNumber = 0 #ord(headerinfo[1935:1936])
    self.h_KeyFramesInFilm = 0 #headerinfo[1936:2056] # set of 30 frames
    self.h_PlayerLocked = 0 # ord(headerinfo[2057:2057])
    self.h_FrameSelectionValid = 0 #ord(headerinfo[2057:2058])
    self.h_FrameofROIStart = 0 #mylong(headerinfo[2058:2062])
    self.h_FrameofROIEnd = 0 #mylong(headerinfo[2062:2066])
    self.h_PlayerLockedROI = 0# ord(headerinfo[2066:2067])
    self.h_PlayerInfinitLoop = 0 #ord(headerinfo[2067:2068])
    self.h_PlayerInitFrame = 0 #mylong(headerinfo[2068:2072])

    self.h_Isoterm0Active = 0 #ord(headerinfo[2072:2073])
    self.h_Isoterm0DLMin = 0 #myint(headerinfo[2073:2075])
    self.h_Isoterm0DLMax = 0 #myint(headerinfo[2075:2077])
    self.h_Isoterm0Color = 0 #headerinfo[2077:2081]

    self.h_Isoterm1Active = 0 #ord(headerinfo[2081:2082])
    self.h_Isoterm1DLMin = 0 #myint(headerinfo[2082:2084])
    self.h_Isoterm1DLMax = 0 #myint(headerinfo[2084:2086])
    self.h_Isoterm1Color = 0 #headerinfo[2086:2090]

    self.h_Isoterm2Active = 0 #ord(headerinfo[2090:2091])
    self.h_Isoterm2DLMin = 0 #myint(headerinfo[2091:2093])
    self.h_Isoterm2DLMax = 0 #myint(headerinfo[2093:2095])
    self.h_Isoterm2Color = 0 #headerinfo[2095:2099]

    self.h_ZeroActive = 0 #ord(headerinfo[2099:2100])
    self.h_ZeroDL = 0 #myint(headerinfo[2100:2102])
    self.h_PaletteWidth = 0 #myint(headerinfo[2102:2104])
    self.h_PaletteFull = 0 #ord(headerinfo[2104:2105])
    self.h_PTRFrameBufferType = 0 #ord(headerinfo[2105:2106]) # 0=word 1=double
    self.h_ThermoElasticity = 0 #headerinfo[2106:2114] # type double (64 bits)
    self.h_DemodulationFrequency = 0 #myfloat(headerinfo[2114:2118])
    self.h_CoordinatesType = 0 #mylong(headerinfo[2118:2122])
    self.h_CoordinatesXorigin = 0 #mylong(headerinfo[2122:2126])
    self.h_CoordinatesYorigin = 0 #mylong(headerinfo[2126:2130])
    self.h_CoordinatesShowOrigin = 0 #ord(headerinfo[2130:2131])
    self.h_AxeColor = 0 #headerinfo[2131:2135]
    self.h_AxeSize = 0 #mylong(headerinfo[2135:2139])
    self.h_AxeValid = 0 #ord(headerinfo[2139:2140])
    self.h_DistanceOffset = 0 #myfloat(headerinfo[2140:2144])
    self.h_HistoEqualizationEnabled = 0 #ord(headerinfo[2144:2145])
    self.h_HistoEqualizationPercent = 0 #myint(headerinfo[2145:2147])
    self.h_CalibrationFileName = '' #headerinfo[2147:2403]
    self.h_PTRTopFrameValid = 0 #ord(headerinfo[2403:2404])
    self.h_SubSampling = 0 #myint(headerinfo[2404:2408])
    self.h_CameraHFlip = 0 #ord(headerinfo[2408:2409])
    self.h_CameraHVFlip = 0 #ord(headerinfo[2409:2410])
    self.h_BBTemp = 0 #myfloat(headerinfo[2410:2414])
    self.h_CaptureWheelIndex = 0 #ord(headerinfo[2414:2415])
    self.h_CaptureFocalIndex = 0 #ord(headerinfo[2415:2416])
    self.h_Reserved7 = '' #headerinfo[2416:3028]
    self.h_Reserved8 = '' #headerinfo[3028:3076]
    self.h_Framatone = 0 #ord(headerinfo[3076:3077]

    # Container for a single frame
    self.data = []
    self.minval = 0
    self.maxval = 0

    # Frame time
    self.h_frameMinute = 0 #FrameHeader[80:81]
    self.h_frameHour = 0 #FrameHeader[81:82]
    self.h_frameSecond = 0 #h_second+(h_thousands+h_hundred)/1000.0

    # detector / FPA temperature
    self.h_detectorTemp = 0.0 #FrameHeader[228:232]
    self.h_sensorTemp4 = 0.0 #FrameHeader[232:236]

# End of header definition

################################################################

def readPTWHeader(ptwfilename):
    """Given a ptw filename, read the header and return the header to caller

    Args:
        | filename (string) with full path to the ptw file.

    Returns:
        | Header (class) containing all PTW header information.

    Raises:
        | No exception is raised.

    Reference:
       h_variables of the header and byte positions are obtained
       from DL002U-D Altair Reference Guide
     """


    # Define the variables holding the header values
    headerinfo = '' #the vector holding the file header
    Header = PTWFrameInfo()

    # Read file to get the header size
    Header.FileName = ptwfilename
    fid = open(ptwfilename,'rb')
    headerinfo = fid.read(16)
    fid.close()
    MainHeaderSize = mylong(headerinfo[11:15])

    # Open file again and read the header information using the header size
    fid = open(ptwfilename,'rb')
    headerinfo = fid.read(MainHeaderSize)

    if sys.version_info[0] > 2:
        Header.h_Signature = headerinfo[0:3].decode('utf-8')
    else:
        Header.h_Signature = headerinfo[0:3]
    if Header.h_Signature == 'AIO': #AGEMA
        Header.h_format = 'agema'
    elif Header.h_Signature == 'CED':
        Header.h_format = 'cedip'
        Header.h_unit = 'dl'

    if sys.version_info[0] > 2:
        Header.h_Version = headerinfo[5:10].decode('utf-8')
    else:
        Header.h_Version = headerinfo[5:10]
    if not Header.h_Version[-1] in string.printable:
        Header.h_Version =  Header.h_Version[:-1]
    Header.h_EofAsciiCode = mybyte(headerinfo[10:11])
    Header.h_MainHeaderSize = mylong(headerinfo[11:15])
    Header.h_FrameHeaderSize = mylong(headerinfo[15:19])
    Header.h_SizeOfOneFrameAndHeader = mylong(headerinfo[19:23])
    Header.h_SizeOfOneFrame = mylong(headerinfo[23:27])
    Header.h_NumberOfFieldInFile = mylong(headerinfo[27:31])
    # Header.h_CurrentFieldNumber = myint(headerinfo[31:35])
    Header.h_CurrentFieldNumber = mylong(headerinfo[31:35])

    #Header.h_FileSaveDate = '' #[35:39] decoded below
    Header.h_FileSaveYear = myint(headerinfo[35:37])
    Header.h_FileSaveDay = ord(headerinfo[37:38])
    Header.h_FileSaveMonth = ord(headerinfo[38:39])

    #Header.h_FileSaveTime = '' #[39:43] decoded below
    Header.h_FileSaveMinute = ord(headerinfo[39:40])
    Header.h_FileSaveHour = ord(headerinfo[40:41])
    Header.h_FileSaveCent = ord(headerinfo[41:42])
    Header.h_FileSaveSecond = ord(headerinfo[42:43])

    Header.h_Millieme = ord(headerinfo[43:44])


    if sys.version_info[0] > 2:
        stripchar = terminateStrOnZero(headerinfo[44:64]).decode('utf-8')[-1]
        Header.h_CameraName = terminateStrOnZero(headerinfo[44:64]).decode('utf-8').rstrip(stripchar)
        Header.h_LensName = terminateStrOnZero(headerinfo[64:84]).decode('utf-8').rstrip(stripchar)
        Header.h_FilterName = terminateStrOnZero(headerinfo[84:104]).decode('utf-8').rstrip(stripchar)
        Header.h_ApertureName = terminateStrOnZero(headerinfo[104:124]).decode('utf-8').rstrip(stripchar)
    else:
        Header.h_CameraName = terminateStrOnZero(headerinfo[44:64])
        Header.h_LensName = terminateStrOnZero(headerinfo[64:84])
        Header.h_FilterName = terminateStrOnZero(headerinfo[84:104])
        Header.h_ApertureName = terminateStrOnZero(headerinfo[104:124])

    Header.h_IRUSBilletSpeed = myfloat(headerinfo[124:128]) # IRUS
    Header.h_IRUSBilletDiameter = myfloat(headerinfo[128:132]) # IRUS
    Header.h_IRUSBilletShape = myint(headerinfo[132:134]) #IRUS
    Header.h_Reserved134 = headerinfo[134:141]
    Header.h_Emissivity = myfloat(headerinfo[141:145])
    Header.h_Ambiant = myfloat(headerinfo[145:149])
    Header.h_Distance = myfloat(headerinfo[149:153])
    Header.h_IRUSInductorCoil = ord(headerinfo[153:154]) # IRUS
    Header.h_IRUSInductorPower = mylong(headerinfo[154:158]) # IRUS
    Header.h_IRUSInductorVoltage = myint(headerinfo[158:160]) # IRUS
    Header.h_IRUSInductorFrequency = mylong(headerinfo[160:164]) # IRUS
    Header.h_Reserved164 = headerinfo[164:169]
    Header.h_IRUSSynchronization = ord(headerinfo[169:170]) # IRUS
    Header.h_AtmTransmission = myfloat(headerinfo[170:174])
    Header.h_ExtinctionCoeficient = myfloat(headerinfo[174:178])
    Header.h_Object = myint(headerinfo[178:180])
    Header.h_Optic = myint(headerinfo[180:182])
    Header.h_Atmo = myint(headerinfo[182:184])
    Header.h_AtmosphereTemp = myfloat(headerinfo[184:188])
    Header.h_CutOnWavelength = myfloat(headerinfo[188:192])
    Header.h_CutOffWavelength = myfloat(headerinfo[192:196])
    Header.h_PixelSize = myfloat(headerinfo[196:200])
    Header.h_PixelPitch = myfloat(headerinfo[200:204])
    Header.h_DetectorApperture = myfloat(headerinfo[204:208])
    Header.h_OpticsFocalLength = myfloat(headerinfo[208:212])
    Header.h_HousingTemperature1 = myfloat(headerinfo[212:216])
    Header.h_HousingTemperature2 = myfloat(headerinfo[216:220])
    if sys.version_info[0] > 2:
        stripchar = terminateStrOnZero(headerinfo[220:231]).decode('utf-8')[-1]
        Header.h_CameraSerialNumber = terminateStrOnZero(headerinfo[220:231]).decode('utf-8').rstrip(stripchar)
    else:
        Header.h_CameraSerialNumber = terminateStrOnZero(headerinfo[220:231])
        
        
    Header.h_Reserved231 = headerinfo[231:239]
    Header.h_DetectorCode1 = myint(headerinfo[239:241])
    Header.h_DetectorCode2 = myint(headerinfo[241:243])
    Header.h_DetectorGain = myint(headerinfo[245:247])

    
    
    Header.h_MinimumLevelThreshold = myint(headerinfo[245:247])
    Header.h_MaximumLevelThreshold = myint(headerinfo[247:249])
    Header.h_EchelleSpecial = myint(headerinfo[277:279])
    Header.h_EchelleUnit = headerinfo[279:289]
    Header.h_EchelleValue = headerinfo[289:357] # 16 float values

    if(Header.h_EchelleSpecial==0):
        Header.h_Units='dl' # [dl T rad]
    else:
        Header.h_Units= Header.h_EchelleUnit # [dl T rad]

    Header.h_LockinGain = myfloat(headerinfo[357:361])
    Header.h_LockinOffset = myfloat(headerinfo[361:365])
    Header.h_HorizontalZoom = myfloat(headerinfo[365:369])
    Header.h_VerticalZoom = myfloat(headerinfo[369:373])

    Header.h_PixelsPerLine = myint(headerinfo[377:379])
    Header.h_LinesPerField = myint(headerinfo[379:381])
    if Header.h_LinesPerField==0:
        Header.h_LinesPerField=128
    if Header.h_PixelsPerLine==0:
        Header.h_PixelsPerLine=128

    Header.h_Rows = Header.h_LinesPerField
    Header.h_Cols = Header.h_PixelsPerLine

    Header.h_cliprect = [0,0,Header.h_Cols-1,Header.h_Rows-1]
    Header.h_lastframe = Header.h_NumberOfFieldInFile
    Header.h_FrameSize = Header.h_FrameHeaderSize + Header.h_Cols * Header.h_Rows * 2

    Header.h_ADDynamic = myint(headerinfo[381:383])
    Header.h_SATIRTemporalFrameDepth = myint(headerinfo[383:385]) # SATIR
    Header.h_SATIRLocationLongitude = myfloat(headerinfo[385:389]) # SATIR
    Header.h_SATIRLocationLatitude = myfloat(headerinfo[389:393]) # SATIR South is negative
    Header.h_SATIRLocationAltitude = myfloat(headerinfo[393:397]) # SATIR
    if sys.version_info[0] > 2:
        Header.h_ExternalSynch = headerinfo[397] # 1=External 0 = Internal
    else:
        Header.h_ExternalSynch = ord(headerinfo[397]) # 1=External 0 = Internal

    Header.h_CEDIPAquisitionPeriod = myfloat(headerinfo[403:407]) # CEDIP seconds
    Header.h_CEDIPIntegrationTime = myfloat(headerinfo[407:411]) # CEDIP seconds
    Header.h_WOLFSubwindowCapability = myint(headerinfo[411:413]) # WOLF
    Header.h_ORIONIntegrationTime = headerinfo[413:437] # ORION (6 values)
    Header.h_ORIONFilterNames = headerinfo[437:557] # ORION 6 fields of 20 chars each
    Header.h_NucTable = myint(headerinfo[557:559])
    Header.h_Reserve6 = headerinfo[559:563]
    if sys.version_info[0] > 2:
        stripchar = terminateStrOnZero(headerinfo[563:1563]).decode('utf-8')[-1]
        Header.h_Comment = terminateStrOnZero(headerinfo[563:1563]).decode('utf-8').rstrip(stripchar)
        stripchar = terminateStrOnZero(headerinfo[1563:1663]).decode('utf-8')[-1]
        Header.h_CalibrationFileName = terminateStrOnZero(headerinfo[1563:1663]).decode('utf-8').rstrip(stripchar)
        stripchar = terminateStrOnZero(headerinfo[1663:1919]).decode('utf-8')[-1]
        Header.h_ToolsFileName = terminateStrOnZero(headerinfo[1663:1919]).decode('utf-8').rstrip(stripchar)
    else:
        Header.h_Comment = terminateStrOnZero(headerinfo[563:1563])
        Header.h_CalibrationFileName = terminateStrOnZero(headerinfo[1563:1663])
        Header.h_ToolsFileName = terminateStrOnZero(headerinfo[1663:1919])

    Header.h_PaletteIndexValid = ord(headerinfo[1919:1920])
    Header.h_PaletteIndexCurrent = myint(headerinfo[1920:1922])
    Header.h_PaletteToggle = ord(headerinfo[1922:1923])
    Header.h_PaletteAGC = ord(headerinfo[1923:1924])
    Header.h_UnitIndexValid = ord(headerinfo[1924:1925])
    Header.h_CurrentUnitIndex = myint(headerinfo[1925:1927])
    if sys.version_info[0] > 2:
        stripchar = terminateStrOnZero(headerinfo[1927:1935]).decode('utf-8', errors='ignore')[-1]
        Header.h_ZoomPosition = terminateStrOnZero(headerinfo[1927:1935]).decode('utf-8', errors='ignore').rstrip(stripchar) # unknown format POINT
        Header.h_KeyFramesInFilm = terminateStrOnZero(headerinfo[1936:2056]).decode('utf-8', errors='ignore').rstrip(stripchar) # set of 30 frames
    else:
        Header.h_ZoomPosition = terminateStrOnZero(headerinfo[1927:1935]) # unknown format POINT
        Header.h_KeyFramesInFilm = terminateStrOnZero(headerinfo[1936:2056]) # set of 30 frames
    Header.h_KeyFrameNumber = ord(headerinfo[1935:1936])
    Header.h_PlayerLocked =  ord(headerinfo[2056:2057])
    Header.h_FrameSelectionValid = ord(headerinfo[2057:2058])
    Header.h_FrameofROIStart = mylong(headerinfo[2058:2062])
    Header.h_FrameofROIEnd = mylong(headerinfo[2062:2066])
    Header.h_PlayerLockedROI =  ord(headerinfo[2066:2067])
    Header.h_PlayerInfinitLoop = ord(headerinfo[2067:2068])
    Header.h_PlayerInitFrame = mylong(headerinfo[2068:2072])

    Header.h_Isoterm0Active = ord(headerinfo[2072:2073])
    Header.h_Isoterm0DLMin = myint(headerinfo[2073:2075])
    Header.h_Isoterm0DLMax = myint(headerinfo[2075:2077])
    Header.h_Isoterm0Color = myRGB(headerinfo[2077:2081])

    Header.h_Isoterm1Active = ord(headerinfo[2081:2082])
    Header.h_Isoterm1DLMin = myint(headerinfo[2082:2084])
    Header.h_Isoterm1DLMax = myint(headerinfo[2084:2086])
    Header.h_Isoterm1Color = myRGB(headerinfo[2086:2090])

    Header.h_Isoterm2Active = ord(headerinfo[2090:2091])
    Header.h_Isoterm2DLMin = myint(headerinfo[2091:2093])
    Header.h_Isoterm2DLMax = myint(headerinfo[2093:2095])
    Header.h_Isoterm2Color = myRGB(headerinfo[2095:2099])

    Header.h_ZeroActive = ord(headerinfo[2099:2100])
    Header.h_ZeroDL = myint(headerinfo[2100:2102])
    Header.h_PaletteWidth = myint(headerinfo[2102:2104])
    Header.h_PaletteFull = ord(headerinfo[2104:2105])
    Header.h_PTRFrameBufferType = ord(headerinfo[2105:2106]) # 0=word 1=double
    Header.h_ThermoElasticity = mydouble(headerinfo[2106:2114]) # type double (64 bits)
    Header.h_DemodulationFrequency = myfloat(headerinfo[2114:2118])
    Header.h_CoordinatesType = mylong(headerinfo[2118:2122])
    Header.h_CoordinatesXorigin = mylong(headerinfo[2122:2126])
    Header.h_CoordinatesYorigin = mylong(headerinfo[2126:2130])
    Header.h_CoordinatesShowOrigin = ord(headerinfo[2130:2131])
    Header.h_AxeColor = myRGB(headerinfo[2131:2135])
    Header.h_AxeSize = mylong(headerinfo[2135:2139])
    Header.h_AxeValid = ord(headerinfo[2139:2140])
    Header.h_DistanceOffset = myfloat(headerinfo[2140:2144])
    Header.h_HistoEqualizationEnabled = ord(headerinfo[2144:2145])
    Header.h_HistoEqualizationPercent = myint(headerinfo[2145:2147])
    if sys.version_info[0] > 2:
        stripchar = terminateStrOnZero(headerinfo[2147:2403]).decode('utf-8', errors='ignore')[-1]
        Header.h_CalibrationFileName = terminateStrOnZero(headerinfo[2147:2403]).decode('utf-8', errors='ignore').rstrip(stripchar)
    else:
        Header.h_CalibrationFileName = terminateStrOnZero(headerinfo[2147:2403])


    Header.h_PTRTopFrameValid = ord(headerinfo[2403:2404])
    # Header.h_SubSampling = myint(headerinfo[2404:2408])
    Header.h_SubSampling = mylong(headerinfo[2404:2408])
    Header.h_CameraHFlip = ord(headerinfo[2408:2409])
    Header.h_CameraHVFlip = ord(headerinfo[2409:2410])
    Header.h_BBTemp = myfloat(headerinfo[2410:2414])
    Header.h_CaptureWheelIndex = ord(headerinfo[2414:2415])
    Header.h_CaptureFocalIndex = ord(headerinfo[2415:2416])
    Header.h_Reserved7 = headerinfo[2416:3028]
    Header.h_Reserved8 = headerinfo[3028:3076]
    Header.h_Framatone = ord(headerinfo[3076:3077])

    # Read the first video frame info, not the data
    # to determine lockin information

    fid.seek(Header.h_MainHeaderSize,0)#,'bof')  %skip main header
    fid.seek(Header.h_FrameHeaderSize,1)#'cof')  %skip frame header
    firstline = fid.read(Header.h_Cols)#, 'uint16')  %read one line

    # look if first line contains lockin information
    if(firstline[1:4]==[1220,3907,1204,2382]):
        Header.h_Lockin=1
        Header.h_Rows=Header.h_Rows-1
        print ('* LOCKIN')
    else:
        Header.h_Lockin=0

    fid.close()

    return Header

################################################################
def GetPTWFrameFromFile(header):
    """From the ptw file, load the frame specified in the header variable
       header.h_framepointer

    Args:
        | header (class object) header of the ptw file, with framepointer set

    Returns:
        | header.data plus newly added information:
          requested frame DL values, dimensions (rows,cols)

    Raises:
        | No exception is raised.
    """

    # for debugging
    #print ('.....Loading frame', header.m_framepointer , 'from', header.m_filename,'.....')
    #print (header.m_cols,'x', header.m_rows, 'data points')

    fid = open(header.FileName,'rb')
    # skip main header
    fid.seek (header.h_MainHeaderSize,0)  #bof

    # for debugging
    #print ('EndHeader =',fid.tell())

    if(header.h_Lockin): # lockin -> skip first line
        fid.seek ((header.h_framepointer-1) * (header.h_FrameSize + 2*header.h_Cols),1)#, 'cof'
    else:
        fid.seek ((header.h_framepointer-1) * (header.h_FrameSize),1)#, 'cof'

    # for debugging
    #print ('StartFrameHeader =',fid.tell())

    #fid.seek(header.m_FrameHeaderSize,1)#,'cof') #skip frame header
    FrameHeader = fid.read(header.h_FrameHeaderSize)

    #Get the frame time
    header.h_frameMinute = ord(FrameHeader[80:81])
    header.h_frameHour = ord(FrameHeader[81:82])
    h_hundred = ord(FrameHeader[82:83])*10
    h_second = ord(FrameHeader[83:84])
    h_thousands = ord(FrameHeader[160:161])
    header.h_frameSecond = h_second+(h_hundred+h_thousands)/1000.0
	#detector FPA temperature
    header.h_detectorTemp = myfloat(FrameHeader[228:232]) 
    header.h_sensorTemp4 = myfloat(FrameHeader[232:236]) 
    if header.h_sensorTemp4 is None:
        header.h_sensorTemp4 = 0.0
    if header.h_detectorTemp is None:
        header.h_detectorTemp = 0.0

    # for debugging
    #print ('Start FrameData at',fid.tell())

    header.data = np.eye(header.h_Cols, header.h_Rows)

    #datapoints = header.m_cols * header.m_rows
    for y in range(header.h_Rows):
        for x in range(header.h_Cols):
            header.data[x][y] = myint(fid.read(2))

    # for debugging
    #print ('Data read',len(header.m_data), 'points')
    #print ('End FrameData at',fid.tell())

    # if a special scale is given then transform the data
    if(header.h_EchelleSpecial):
        low = min(header.h_EchelleScaleValue)
        high = max(header.h_EchelleScaleValue)
        header.data = header.data * (high-low)/ 2.0**16 + low
        #clear low high
    if(header.h_Lockin): # lockin -> skip first line
        header.h_cliprect = [0,1,header.h_Cols-1,header.h_Rows]

    header.h_minval = header.data.min()
    header.h_maxval = header.data.max()

    # for debugging
    #print ('DL values min', header.m_minval)
    #print ('DL values max', header.m_maxval)

    fid.close()  #close file
    return header

################################################################
def getPTWFrame (header, frameindex):
    """Retrieve a single PTW frame, given the header and frame index

    This routine also stores the data array as part of the header. This may
    change - not really needed to have both a return value and header stored
    value for the DL valueheader. This for a historical reason due to the way
    GetPTWFrameFromFile was written.  

    The contents of the header is changed (new information added: frame time 
    and detector temperature).  The header is returned from the function to
    make it explicit that the contents have changed from the header passed into
    the function.

    Args:
        | header (class object)
        | frameindex (integer): The frame to be extracted

    Returns:
        | header.data (np.ndarray): requested frame DL values, dimensions (rows,cols)
        | header (class): updated header now with frame time and FPA temperature

    Raises:
        | No exception is raised.
    """

    # Check if this is  a cedip file
    errorresult = np.asarray([0])
    if header.h_format!='cedip':
        print('ReadJade Error: file format is not supported')
        return errorresult
    if (frameindex <= header.h_lastframe):
        if frameindex>0:
            header.h_framepointer = frameindex
            header = GetPTWFrameFromFile(header)
        else:
            print ('frameindex smaller than 0')
            return errorresult
    else:                           # frameindex exceeds no of frames
        print ('ReadJade Error: cannot load frame. Frameindex exceeds sequence length.')
        return errorresult

    return header.data.conj().transpose(), header


################################################################
def getPTWFrames (header, loadFrames=[]):
    """Retrieve a number of PTW frames, given in a list of frameheader.
    The function returns the image data as well as the file and image header 
    data (time and FPA temperature) valid for the particular frame.
    The frame header data is written in the same class as is the file
    header, in order to keep all the information together for the frame.

    Args:
        | header (class object)
        | loadFrames ([int]): List of indices for frames to be extracted

    Returns:
        | data (np.ndarray): requested image frame DL values, dimensions (frames,rows,cols)
        | fheaders (object): requested image frame header values

    Raises:
        | No exception is raised.
    """

    fheaders = []

    # error checking on inputs
    errorresult = np.asarray([0])
    # Check if this is  a cedip file
    if header.h_format!='cedip':
        print('getPTWFrames Error: file format is not supported')
        return errorresult,None
    #check for legal frame index values
    npFrames = np.asarray(loadFrames)
    if np.any( npFrames < 1 ) or np.any ( npFrames > header.h_lastframe ):
        print('getPTWFrames Error: at least one requested frame not in file')
        print('legal frames for this file are: {0} to {1}'.format(1,header.h_lastframe))
        return errorresult, None

    data, headerx = getPTWFrame (header, loadFrames[0])
    fheaders.append(headerx)

    for frame in loadFrames[1:]:
        datax, headerx = getPTWFrame (header, frame)
        data = np.concatenate((data, datax))
        fheaders.append(headerx)

    rows = header.h_Rows
    cols = header.h_Cols

    return data.reshape(len(loadFrames), rows ,cols), fheaders


################################################################
def showHeader(Header):
    """Utility function to print the PTW header information to stdout

    Args:
        | header (class object) ptw file header structure

    Returns:
        | None

    Raises:
        | No exception is raised.
    """

    print('{} version {}'.format(Header.h_Signature, Header.h_Version))
    print('Main Header Size {}'.format(Header.h_MainHeaderSize))
    print('Frame Header Size {}'.format(Header.h_FrameHeaderSize))
    print('Frame + Frame Header Size {}'.format(Header.h_SizeOfOneFrameAndHeader))
    print('Frame Size {}'.format(Header.h_SizeOfOneFrame))
    print('Number of Frames {}'.format(Header.h_NumberOfFieldInFile))
    #print Header.h_CurrentFieldNumber

    print('Year {} Month {} Day {}'.format(Header.h_FileSaveYear, Header.h_FileSaveMonth, 
        Header.h_FileSaveDay))
    print('( {} / {} / {} )'.format(str(Header.h_FileSaveYear).zfill(2), 
        str(Header.h_FileSaveMonth).zfill(2), str(Header.h_FileSaveDay).zfill(2)))
    print('Hour {} Minute {} Second {}'.format(Header.h_FileSaveHour,Header.h_FileSaveMinute,
        Header.h_FileSaveSecond))
    print('( {} : {} : {} )'.format(str(Header.h_FileSaveHour).zfill(2),
        str(Header.h_FileSaveMinute).zfill(2),str(Header.h_FileSaveSecond).zfill(2)))

    #print Header.h_Millieme
    print ('Camera Name {}'.format(Header.h_CameraName))
    print ('Detector Code1 {}'.format(Header.h_DetectorCode1))
    print ('Detector Code2 {}'.format(Header.h_DetectorCode2))
    print ('Detector Gain {}'.format(Header.h_DetectorGain))

    print ('Lens {}'.format(Header.h_LensName))
    print ('Filter {}'.format(Header.h_FilterName))
    print ('Aperture Name {}'.format( Header.h_ApertureName))
    if Header.h_Signature == 'IRUS':
        print ('{}'.format(Header.h_IRUSBilletSpeed))
        print ('{}'.format(Header.h_IRUSBilletDiameter))
        print ('{}'.format(Header.h_IRUSBilletShape))
    print ('Emissivity {:.6f}'.format(Header.h_Emissivity))
    print ('Ambient Temperature {:.6f} (K)'.format(Header.h_Ambiant))
    print ('Ambient Temperature {:.6f} (degC)'.format(Header.h_Ambiant-273.15))
    print ('Distance to target {}'.format(Header.h_Distance))
    if Header.h_Signature == 'IRUS':
        print ('{}'.format(Header.h_IRUSInductorCoil))
        print ('{}'.format(Header.h_IRUSInductorPower))
        print ('{}'.format(Header.h_IRUSInductorVoltage))
        print ('{}'.format(Header.h_IRUSInductorFrequency))
        print ('{}'.format(Header.h_IRUSSynchronization))
    print ('Atm Transmission {}'.format(Header.h_AtmTransmission))
    print ('Ext Coef {}'.format(Header.h_ExtinctionCoeficient))
    print ('Target {}'.format(Header.h_Object))
    print ('Optic {}'.format(Header.h_Optic))
    print ('Atmo {}'.format(Header.h_Atmo))
    print ('Atm Temp {:.6f}'.format(Header.h_AtmosphereTemp))
    print ('Cut on Wavelength {:.6f}'.format(Header.h_CutOnWavelength))
    print ('Cut off Wavelength {:.6f}'.format(Header.h_CutOffWavelength))
    print ('PixelSize {}'.format(Header.h_PixelSize))
    print ('PixelPitch {}'.format(Header.h_PixelPitch))
    print ('Detector Apperture {}'.format(Header.h_DetectorApperture))
    print ('Optic Focal Length {}'.format(Header.h_OpticsFocalLength))
    print ('Housing Temp1 {:.6f} (K)'.format(Header.h_HousingTemperature1))
    print ('Housing Temp2 {:.6f} (K)'.format(Header.h_HousingTemperature2))

    print ('Sensor Temp4 {:.6f} (K)'.format(Header.h_sensorTemp4))
    print ('Detector/FPA Temp {:.6f} (K)'.format(Header.h_detectorTemp))
    
    print ('Camera Serial Number {}'.format(Header.h_CameraSerialNumber))
    print ('Min Threshold {}'.format(Header.h_MinimumLevelThreshold))
    print ('Max Threshold {}'.format(Header.h_MaximumLevelThreshold))
    #print Header.h_EchelleSpecial
    #print Header.h_EchelleUnit
    #print Header.h_EchelleValue
    print ('Gain {}'.format(Header.h_LockinGain))
    print ('Offset {}'.format(Header.h_LockinOffset))
    print ('HZoom {}'.format(Header.h_HorizontalZoom))
    print ('VZoom {}'.format(Header.h_VerticalZoom))
    print ('Field {}'.format(Header.h_PixelsPerLine,'X',Header.h_LinesPerField))
    print ('AD converter {} bit'.format(Header.h_ADDynamic))




    if Header.h_Signature == 'SATIR':
        print ('{}'.format(Header.h_SATIRTemporalFrameDepth))
        print ('{}'.format(Header.h_SATIRLocationLongitude))
        print ('{}'.format(Header.h_SATIRLocationLatitude))
        print ('{}'.format(Header.h_SATIRLocationAltitude))
    if Header.h_ExternalSynch:
        print ('Ext Sync ON')
    else:
        print ('Ext Sync OFF')
    print('Header.h_Signature = {}'.format(Header.h_Signature))
    if Header.h_Signature == 'CED':
        print ('CEDIP Period {:.6f} Hz'.format(1./Header.h_CEDIPAquisitionPeriod))
        print ('CEDIP Integration {:.6f} msec'.format(Header.h_CEDIPIntegrationTime*1000))
    if Header.h_Signature == 'WOLF':
        print ( '{}'.format(Header.h_WOLFSubwindowCapability))
    if Header.h_Signature == 'ORI':
        print ( '{}'.format(Header.h_ORIONIntegrationTime))
        print ( '{}'.format(Header.h_ORIONFilterNames))
    print ('NUC {}'.format(Header.h_NucTable))
    #print Header.h_Reserve6
    print ('Comment {}'.format(Header.h_Comment))

    print ('Calibration File Name {}'.format(Header.h_CalibrationFileName))

    print ('Tools File Name {}'.format(Header.h_ToolsFileName))

    print ('Palette Index {}'.format(Header.h_PaletteIndexValid))
    print ('Palette Current {}'.format(Header.h_PaletteIndexCurrent))
    print ('Palette Toggle {}'.format(Header.h_PaletteToggle))
    print ('Palette AGC {}'.format(Header.h_PaletteAGC))
    print ('Unit Index {}'.format(Header.h_UnitIndexValid))
    print ('Current Unit Index {}'.format(Header.h_CurrentUnitIndex))
    print ('Zoom Pos {}'.format(Header.h_ZoomPosition))
    print ('Key Framenum {}'.format(Header.h_KeyFrameNumber))
    print ('Num Keyframes {}'.format(Header.h_KeyFramesInFilm))
    print ('Player lock {}'.format(Header.h_PlayerLocked))
    print ('Frame Select {}'.format(Header.h_FrameSelectionValid))
    print ('ROI Start {}'.format(Header.h_FrameofROIStart))
    print ('ROI Stop {}'.format(Header.h_FrameofROIEnd))
    print ('Player inf loop {}'.format(Header.h_PlayerInfinitLoop))
    print ('Player Init Frame {}'.format(Header.h_PlayerInitFrame))

    print ('Isoterm0 {}'.format(Header.h_Isoterm0Active))
    print ('Isoterm0 DL Min {}'.format(Header.h_Isoterm0DLMin))
    print ('Isoterm0 DL Max {}'.format(Header.h_Isoterm0DLMax))
    print ('Isoterm0 Color RGB {}'.format(Header.h_Isoterm0Color))

    print ('Isoterm1 {}'.format(Header.h_Isoterm1Active))
    print ('Isoterm1 DL Min {}'.format(Header.h_Isoterm1DLMin))
    print ('Isoterm1 DL Max {}'.format(Header.h_Isoterm1DLMax))
    print ('Isoterm1 Color RGB {}'.format(Header.h_Isoterm1Color))

    print ('Isoterm2 {}'.format(Header.h_Isoterm2Active))
    print ('Isoterm2 DL Min {}'.format(Header.h_Isoterm2DLMin))
    print ('Isoterm2 DL Max {}'.format(Header.h_Isoterm2DLMax))
    print ('Isoterm2 Color RGB {}'.format(Header.h_Isoterm2Color))

    print ('Zero {}'.format(Header.h_ZeroActive))
    print ('Zero DL {}'.format(Header.h_ZeroDL))
    print ('Palette Width {}'.format(Header.h_PaletteWidth))
    print ('PaletteF Full {}'.format(Header.h_PaletteFull))
    print ('PTR Frame Buffer type {}'.format(Header.h_PTRFrameBufferType))
    print ('Thermoelasticity {}'.format(Header.h_ThermoElasticity))
    print ('Demodulation {}'.format(Header.h_DemodulationFrequency))
    print ('Coordinate Type {}'.format(Header.h_CoordinatesType))
    print ('X Origin {}'.format(Header.h_CoordinatesXorigin))
    print ('Y Origin {}'.format(Header.h_CoordinatesYorigin))
    print ('Coord Show Orig {}'.format(Header.h_CoordinatesShowOrigin))
    print ('Axe Colour RGB {}'.format(Header.h_AxeColor))
    print ('Axe Size {}'.format(Header.h_AxeSize))
    print ('Axe Valid {}'.format(Header.h_AxeValid))
    print ('Distance offset {}'.format(Header.h_DistanceOffset))
    print ('Histogram {}'.format(Header.h_HistoEqualizationEnabled))
    print ('Histogram % {}'.format(Header.h_HistoEqualizationPercent))

    print ('Calibration File Name {}'.format(Header.h_CalibrationFileName))

    print ('PTRFrame Valid {}'.format(Header.h_PTRTopFrameValid))
    print ('Subsampling {}'.format(Header.h_SubSampling))
    print ('Camera flip H {}'.format(Header.h_CameraHFlip))
    print ('Camera flip V {}'.format(Header.h_CameraHVFlip))
    print ('BB Temp {}'.format(Header.h_BBTemp))
    print ('Capture Wheel Index {}'.format(Header.h_CaptureWheelIndex))
    print ('Capture Wheel Focal Index {}'.format(Header.h_CaptureFocalIndex))
    #print Header.h_Reserved7
    #print Header.h_Reserved8
    #print Header.h_Framatone


In [2]:
import cv2
import pandas as pd
import numpy as np
from typing import Union, List
import os
import scipy.signal as sig
from typing import List, Dict, Tuple, Union
from scipy.optimize import linear_sum_assignment
from copy import deepcopy
from tqdm import tqdm
import time
from math import atan2, cos, sin, radians, degrees, pi
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

## Конвертация .ptw в np.array

In [3]:
# Простые относительные пути
ptw_dir = 'ptw/'
output_dir = 'num_py_convert_new/'

# Создаем папку для выходных данных
os.makedirs(output_dir, exist_ok=True)

ptws = [f for f in os.listdir(ptw_dir) if f.endswith('.ptw')]

print(f"Найдено {len(ptws)} PTW файлов")

for ptw in tqdm(ptws, total=len(ptws)):
    # Формируем имя выходного файла
    output_path = os.path.join(output_dir, f"thermogram_{ptw.replace('.ptw', '')}.npy")
    
    # Если файл уже существует, пропускаем
    if os.path.exists(output_path):
        print(f"{output_path} уже существует, пропускаем")
        continue

    ptw_path = os.path.join(ptw_dir, ptw)
    
    header = readPTWHeader(ptw_path)
    frames = getPTWFrames(header, range(1, header.h_lastframe + 1))[0]
    
    # Сохраняем только если файла еще не было
    np.save(output_path, frames.astype(np.int16))


Найдено 34 PTW файлов


100%|██████████| 34/34 [00:00<00:00, 13837.21it/s]

num_py_convert_new/thermogram_40.npy уже существует, пропускаем
num_py_convert_new/thermogram_31.npy уже существует, пропускаем
num_py_convert_new/thermogram_17.npy уже существует, пропускаем
num_py_convert_new/thermogram_20.npy уже существует, пропускаем
num_py_convert_new/thermogram_42.npy уже существует, пропускаем
num_py_convert_new/thermogram_18.npy уже существует, пропускаем
num_py_convert_new/thermogram_30.npy уже существует, пропускаем
num_py_convert_new/thermogram_10.npy уже существует, пропускаем
num_py_convert_new/thermogram_14.npy уже существует, пропускаем
num_py_convert_new/thermogram_26.npy уже существует, пропускаем
num_py_convert_new/thermogram_35.npy уже существует, пропускаем
num_py_convert_new/thermogram_36.npy уже существует, пропускаем
num_py_convert_new/thermogram_39.npy уже существует, пропускаем
num_py_convert_new/thermogram_32.npy уже существует, пропускаем
num_py_convert_new/thermogram_23.npy уже существует, пропускаем
num_py_convert_new/thermogram_19.npy уже

## Задаем файл с кадрами

In [4]:
# Номер термограммы
thermogram_id = 7

output_path = f'num_py_convert_new/filtered/thermogram_{thermogram_id}_filtered.npy'
file_path = f'num_py_convert_new/thermogram_{thermogram_id}.npy'

frames = np.load(file_path)

In [5]:
def show_video_realtime(frames: Union[np.ndarray, List[np.ndarray]], 
                       fps: int = 25, 
                       window_name: str = "Video Viewer",
                       colormap: int = cv2.COLORMAP_JET,
                       normalize: bool = True,
                       global_normalization: bool = True):
    """
    Показывает numpy массив кадров в виде видео с возможностью пролистывания
    
    Parameters:
    -----------
    frames : np.ndarray или list of np.ndarray
        Массив кадров формы (N, H, W) или список массивов (H, W)
    fps : int
        Количество кадров в секунду
    window_name : str
        Название окна
    colormap : int
        OpenCV colormap (cv2.COLORMAP_*)
    normalize : bool
        Нормализовать ли изображение для отображения
    global_normalization : bool
        Использовать глобальную нормализацию по всем кадрам
    """
    # Преобразуем в список numpy массивов если нужно
    if isinstance(frames, np.ndarray):
        if frames.ndim == 3:
            frame_list = [frames[i] for i in range(frames.shape[0])]
        else:
            raise ValueError("Input array must be 3D (frames, height, width)")
    else:
        frame_list = frames
    
    total_frames = len(frame_list)
    delay = int(1000 / fps)
    current_frame = 0
    paused = False
    
    # Предварительная нормализация если включена глобальная
    if normalize and global_normalization:
        # Собираем все значения для глобальной нормализации
        all_values = np.concatenate([f.flatten() for f in frame_list])
        min_val = np.percentile(all_values, 1)
        max_val = np.percentile(all_values, 99)
        max_val = max(max_val, min_val + 1e-6)
        print(f"Глобальная нормализация: min={min_val:.2f}, max={max_val:.2f}")
    else:
        min_val = max_val = None
    
    # Создаем окно
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 750, 750)
    
    # Функция для подготовки кадра к отображению
    def prepare_frame(frame_idx: int) -> np.ndarray:
        frame = frame_list[frame_idx]
        
        if normalize:
            if global_normalization:
                # Используем предварительно вычисленные глобальные значения
                frame_clip = np.clip(frame, min_val, max_val)
                frame_norm = ((frame_clip - min_val) / (max_val - min_val) * 255).astype(np.uint8)
            else:
                # Локальная нормализация для каждого кадра
                frame_min = np.percentile(frame, 1)
                frame_max = np.percentile(frame, 99)
                frame_max = max(frame_max, frame_min + 1e-6)
                frame_clip = np.clip(frame, frame_min, frame_max)
                frame_norm = ((frame_clip - frame_min) / (frame_max - frame_min) * 255).astype(np.uint8)
        else:
            # Просто приводим к uint8
            frame_norm = np.clip(frame, 0, 255).astype(np.uint8)
        
        # Применяем colormap если изображение одноканальное
        if len(frame_norm.shape) == 2 or (len(frame_norm.shape) == 3 and frame_norm.shape[2] == 1):
            frame_display = cv2.applyColorMap(frame_norm, colormap)
        else:
            frame_display = frame_norm
        
        return frame_display
    
    print("Управление:")
    print("  SPACE - пауза/продолжить")
    print("  A/←    - предыдущий кадр")
    print("  D/→    - следующий кадр")
    print("  W/↑    - +10 кадров")
    print("  S/↓    - -10 кадров")
    print("  Home   - первый кадр")
    print("  End    - последний кадр")
    print("  R      - сбросить воспроизведение")
    print("  +/-    - изменить скорость (FPS)")
    print("  C      - переключить colormap")
    print("  N      - переключить нормализацию")
    print("  G      - переключить глобальную/локальную нормализацию")
    print("  ESC    - выход")
    
    # Доступные colormaps
    available_colormaps = [
        cv2.COLORMAP_JET,
        cv2.COLORMAP_HOT,
        cv2.COLORMAP_COOL,
        cv2.COLORMAP_SPRING,
        cv2.COLORMAP_SUMMER,
        cv2.COLORMAP_AUTUMN,
        cv2.COLORMAP_WINTER,
        cv2.COLORMAP_BONE,
        cv2.COLORMAP_OCEAN,
        cv2.COLORMAP_RAINBOW
    ]
    current_colormap_idx = 0
    colormap_names = ["JET", "HOT", "COOL", "SPRING", "SUMMER", "AUTUMN", "WINTER", "BONE", "OCEAN", "RAINBOW"]
    
    while True:
        try:
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        except cv2.error:
            break
        
        # Подготавливаем и отображаем текущий кадр
        frame_display = prepare_frame(current_frame)
        
        # Добавляем информацию на кадр
        text_lines = [
            f"Frame: {current_frame + 1}/{total_frames}",
            f"FPS: {fps} {'[PAUSED]' if paused else ''}",
            f"Colormap: {colormap_names[current_colormap_idx]}",
            f"Norm: {'Global' if global_normalization else 'Local'}",
            f"Size: {frame_list[current_frame].shape}"
        ]
        
        for i, text in enumerate(text_lines):
            y_position = 30 + i * 25
            cv2.putText(frame_display, text, (10, y_position), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
            # Тень текста для лучшей читаемости
            cv2.putText(frame_display, text, (11, y_position + 1), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, cv2.LINE_AA)
        
        # Прогресс-бар
        bar_width = frame_display.shape[1] - 20
        bar_height = 10
        bar_x, bar_y = 10, frame_display.shape[0] - 20
        progress = (current_frame + 1) / total_frames
        
        # Фон прогресс-бара
        cv2.rectangle(frame_display, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (50, 50, 50), -1)
        # Заполненная часть
        cv2.rectangle(frame_display, (bar_x, bar_y), (bar_x + int(bar_width * progress), bar_y + bar_height), (0, 255, 0), -1)
        # Рамка
        cv2.rectangle(frame_display, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (255, 255, 255), 1)
        
        cv2.imshow(window_name, frame_display)
        
        # Обработка клавиш
        key = cv2.waitKey(0 if paused else delay) & 0xFF
        
        if key == 27:  # ESC
            break
        elif key == ord(' '):  # Пробел - пауза
            paused = not paused
        elif key in [ord('a'), ord('A'), 81]:  # Назад (A или ←)
            current_frame = max(0, current_frame - 1)
            paused = True
        elif key in [ord('d'), ord('D'), 83]:  # Вперёд (D или →)
            current_frame = min(total_frames - 1, current_frame + 1)
            paused = True
        elif key in [ord('w'), ord('W'), 82]:  # +10 кадров (W или ↑)
            current_frame = min(total_frames - 1, current_frame + 10)
            paused = True
        elif key in [ord('s'), ord('S'), 84]:  # -10 кадров (S или ↓)
            current_frame = max(0, current_frame - 10)
            paused = True
        elif key == 80:  # Home - первый кадр
            current_frame = 0
            paused = True
        elif key == 87:  # End - последний кадр
            current_frame = total_frames - 1
            paused = True
        elif key == ord('r'):  # R - сброс
            current_frame = 0
            paused = False
        elif key == ord('+'):  # + - увеличить FPS
            fps = min(60, fps + 5)
            delay = int(1000 / fps)
        elif key == ord('-'):  # - - уменьшить FPS
            fps = max(1, fps - 5)
            delay = int(1000 / fps)
        elif key == ord('c'):  # C - сменить colormap
            current_colormap_idx = (current_colormap_idx + 1) % len(available_colormaps)
            colormap = available_colormaps[current_colormap_idx]
        elif key == ord('n'):  # N - переключить нормализацию
            normalize = not normalize
        elif key == ord('g'):  # G - переключить глобальную/локальную нормализацию
            global_normalization = not global_normalization
            if normalize and global_normalization:
                # Пересчитываем глобальные значения
                all_values = np.concatenate([f.flatten() for f in frame_list])
                min_val = np.percentile(all_values, 1)
                max_val = np.percentile(all_values, 99)
                max_val = max(max_val, min_val + 1e-6)
        elif not paused:
            current_frame = (current_frame + 1) % total_frames
    
    cv2.destroyAllWindows()
    print("Просмотр завершен")



In [6]:
def view_npy_video(file_path, fps=25):
    """
    Загружает и просматривает видео из .npy файла
    
    Args:
        file_path: путь до .npy файла
        fps: кадров в секунду для воспроизведения
    """
    try:
        # Проверяем существование файла
        if not os.path.exists(file_path):
            print(f"Ошибка: файл {file_path} не найден")
            return
        
        # Загружаем данные
        frames = np.load(file_path)
        
        # Выводим информацию о видео
        print(f"Загружено {len(frames)} кадров из {os.path.basename(file_path)}")
        print(f"Размер кадра: {frames[0].shape}")
        print(f"Тип данных: {frames.dtype}")
        print(f"Диапазон значений: [{frames.min():.3f}, {frames.max():.3f}]")
        
        # Просматриваем видео
        window_name = f"Video: {os.path.basename(file_path)}"
        show_video_realtime(frames, fps=fps, window_name=window_name)
        
    except Exception as e:
        print(f"Ошибка при загрузке или воспроизведении видео: {e}")

In [7]:
# Просмотр конкретного файла
# view_npy_video('file_path, fps=25)

In [8]:
def min_loc_LoG(img, k_size = 9, sigma = 1.8):
    """
    Perform min-loc-LoG filtering of grayscale image img
    Sungho K. Min-local-LoG Filter for Detecting Small Targets in 
    Cluttered Background // Electronics Letters. 
    – 2011. – Vol. 47. – № 2. – P. 105-106. DOI: 10.1049/el.2010.2066.

    sigma - std of gaussian
    k_size - size of kernel
    """
    x = np.arange(k_size).reshape(1, k_size)
    y = np.arange(k_size).reshape(k_size, 1)
    # generate fE (positive X)
    fE = (1 - (x**2) / (sigma**2)) * np.exp(- (x**2) / (2*(sigma**2)))
    fE[fE > 0] = fE[fE > 0] / fE[fE > 0].sum()
    fE[fE < 0] = fE[fE < 0] / (-fE[fE < 0].sum())
    # generate fS (positive Y)
    fS = (1 - (y**2) / (sigma**2)) * np.exp(- (y**2) / (2*(sigma**2)))
    fS[fS > 0] = fS[fS > 0] / fS[fS > 0].sum()
    fS[fS < 0] = fS[fS < 0] / (-fS[fS < 0].sum())
    # generate fW
    x = - np.fliplr(x)
    fW = (1 - (x**2) / (sigma**2)) * np.exp(- (x**2) / (2*(sigma**2)))
    fW[fW > 0] = fW[fW > 0] / fW[fW > 0].sum()
    fW[fW < 0] = fW[fW < 0] / (-fW[fW < 0].sum())
    # generate fN
    y = - np.flipud(y)
    fN = (1 - (y**2) / (sigma**2)) * np.exp(- (y**2) / (2*(sigma**2)))
    fN[fN > 0] = fN[fN > 0] / fN[fN > 0].sum()
    fN[fN < 0] = fN[fN < 0] / (-fN[fN < 0].sum())
    # perform 2D convolution with kernels
    def move(img, x, y):
        move_matrix = np.float32([[1, 0, x], [0, 1, y]])
        dimensions = (img.shape[1], img.shape[0])
        return cv2.warpAffine(img, move_matrix, dimensions)

    Ie = sig.convolve2d(move(img, 4, 0), fE, mode = "same")
    Is = sig.convolve2d(move(img, 0, 4), fS, mode = "same")
    Iw = sig.convolve2d(move(img, -4, 0), fW, mode = "same")
    In = sig.convolve2d(move(img, 0, -4), fN, mode = "same")
    f = np.dstack((Ie, Is, Iw, In))
    fmap = np.min(f, axis = 2)
    #return (fmap / fmap.max() * 255).astype(np.uint8)
    return fmap

In [9]:
def min_loc_LoG_video(frames: np.ndarray, k_size: int = 9, sigma: float = 1.8) -> np.ndarray:
    """
    Apply min-loc-LoG filtering to each frame with progress bar
    
    Args:
        frames: numpy array of shape (N, H, W) - video frames
        k_size: size of kernel
        sigma: std of gaussian
    
    Returns:
        filtered_frames: numpy array of shape (N, H, W) - filtered video
    """
    filtered_frames = []
    
    for i in tqdm(range(frames.shape[0]), desc="Processing frames"):
        frame_result = min_loc_LoG(frames[i], k_size, sigma)
        filtered_frames.append(frame_result)
    
    return np.array(filtered_frames)



## Применение фильтра к np.array для обнаружения капель, перезапись фильтрованного np.array

In [10]:
# Проверка существования исходного файла
if not os.path.exists(file_path):
    print(f"Файл {file_path} не найден!")
else:
    # Запрос подтверждения на обработку / перезапись
    proceed = True
    if os.path.exists(output_path):
        answer = input(f"Файл {output_path} уже существует. Перезаписать? (y/n): ").strip().lower()
        if answer != 'y':
            proceed = False
            print("Операция отменена пользователем.")
    
    if proceed:
        # Загрузка и обработка
        frames = np.load(file_path)
        filtered_video = min_loc_LoG_video(frames, k_size=9, sigma=1.8)
        
        # Сохранение результата
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        np.save(output_path, filtered_video)
        print(f"Отфильтрованное видео сохранено в: {output_path}")
        print(f"Размер: {filtered_video.shape}, тип данных: {filtered_video.dtype}")



Операция отменена пользователем.


## Визуализация, отметки на графиках

In [11]:
def draw_triangle(img, cx, cy, size, color):
    """
    Простой равносторонний треугольник по центру (cx, cy)
    """
    cx, cy = int(cx), int(cy)
    h = size

    pts = np.array([
        [cx,        cy - h],
        [cx - h//2, cy + h//2],
        [cx + h//2, cy + h//2]
    ], np.int32)

    cv2.polylines(img, [pts], isClosed=True, color=color, thickness=1)


In [12]:
def get_corridor_segment(first_center, end_center, corridor_width=20):
    """
    Возвращает координаты четырех углов сегмента коридора от first_center до end_center.
    """
    vec = end_center - first_center
    length = np.linalg.norm(vec)
    if length < 1e-6:
        direction = np.array([0, 1], dtype=float)
    else:
        direction = vec / length
    perp = np.array([-direction[1], direction[0]])
    half_w = corridor_width / 2

    corners = np.array([
        first_center + perp * half_w,
        first_center - perp * half_w,
        end_center - perp * half_w,
        end_center + perp * half_w
    ], dtype=int)
    return corners

In [13]:
def show_video_with_detections(frames: np.ndarray,
                               cleaned_spatters: list,
                               static_spatters: list,
                               welding_boxes: list,
                               welding_ellipses: list,
                               hot_stripes_list: list = None,  # полосы для всех кадров
                               fps: int = 25,
                               window_name: str = "Welding Analysis",
                               colormap: int = cv2.COLORMAP_JET,
                               normalize: bool = True,
                               global_normalization: bool = True,
                               dead_zone_radius: int = 50,
                               corridor_width: int = 20,
                               lookahead: int = 5):

    if frames.ndim != 3:
        raise ValueError("Input array must be 3D")

    frame_list = [frames[i] for i in range(frames.shape[0])]
    total_frames = len(frame_list)
    delay = int(1000 / fps)
    current_frame = 0
    paused = False

    available_colormaps = [
        cv2.COLORMAP_JET, cv2.COLORMAP_HOT, cv2.COLORMAP_COOL,
        cv2.COLORMAP_TURBO, cv2.COLORMAP_VIRIDIS
    ]
    current_colormap_idx = available_colormaps.index(colormap) if colormap in available_colormaps else 0
    colormap = available_colormaps[current_colormap_idx]

    if normalize and global_normalization:
        all_values = np.concatenate([f.flatten() for f in frame_list])
        min_val = np.percentile(all_values, 1)
        max_val = np.percentile(all_values, 99)
        max_val = max(max_val, min_val + 1e-6)
    else:
        min_val = max_val = None

    # ----------------------------
    # FLAGS TOGGLERS
    # ----------------------------
    show_box = False
    show_ellipse = True
    show_static = False
    show_corridor = False
    show_hot_stripes = True  # клавиша 5 для отображения горячих полос

    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 1100, 900)
    cv2.imshow(window_name, np.zeros((20, 20), dtype=np.uint8))
    cv2.waitKey(60)

    def prepare_frame_with_detections(frame_idx: int) -> np.ndarray:
        frame = frame_list[frame_idx]

        # --- нормализация ---
        if normalize:
            if global_normalization:
                frame_clip = np.clip(frame, min_val, max_val)
                frame_norm = ((frame_clip - min_val) / (max_val - min_val) * 255).astype(np.uint8)
            else:
                f_min = np.percentile(frame, 1)
                f_max = np.percentile(frame, 99)
                f_max = max(f_max, f_min + 1e-6)
                frame_clip = np.clip(frame, f_min, f_max)
                frame_norm = ((frame_clip - f_min) / (f_max - f_min) * 255).astype(np.uint8)
        else:
            frame_norm = np.clip(frame, 0, 255).astype(np.uint8)

        frame_display = cv2.applyColorMap(frame_norm, colormap)

        # ----------------------------
        # Рисуем горячие полосы через эллипсы
        # ----------------------------
        if hot_stripes_list is not None and show_hot_stripes and hot_stripes_list[frame_idx]:
            for stripe in hot_stripes_list[frame_idx]:
                cx, cy, MAJOR, MINOR, angle = stripe
                if MAJOR < 2 or MINOR < 2:
                    continue
                center = (int(cx), int(cy))
                axes = (max(int(MAJOR / 2), 1), max(int(MINOR / 2), 1))
                angle_deg = np.degrees(angle) % 360
                cv2.ellipse(frame_display, center, axes, angle_deg, 0, 360, (0, 255, 255), 2)

        # ----------------------------
        # накопительный мертвый коридор
        # ----------------------------
        if show_corridor:
            corridor_segments = []
            first_center = None
            for idx in range(frame_idx + 1):
                ell = welding_ellipses[idx]
                if ell is not None:
                    center = np.array(ell[0], dtype=float)
                    if first_center is not None:
                        segment = get_corridor_segment(first_center, center, corridor_width)
                        corridor_segments.append(segment)
                    first_center = center

            for seg in corridor_segments:
                cv2.polylines(frame_display, [seg], isClosed=True, color=(0, 255, 255), thickness=2)

        # ----------------------------
        # зона сварки
        # ----------------------------
        weld_box = welding_boxes[frame_idx]
        if weld_box is not None:
            if show_box:
                cx, cy, w, h = map(int, weld_box)
                x1, y1 = cx - w // 2, cy - h // 2
                x2, y2 = cx + w // 2, cy + h // 2
                cv2.rectangle(frame_display, (x1, y1), (x2, y2), (0, 255, 255), 2)

            if show_ellipse and welding_ellipses[frame_idx] is not None:
                try:
                    cv2.ellipse(frame_display, welding_ellipses[frame_idx], (0, 255, 0), 2)
                    (xc, yc), (_, _), _ = welding_ellipses[frame_idx]
                    cv2.circle(frame_display, (int(xc), int(yc)), dead_zone_radius, (0, 255, 255), 1)
                except:
                    pass

        # ----------------------------
        # обычные брызги
        # ----------------------------
        for sp in cleaned_spatters[frame_idx]:
            cx, cy, w, h = sp.astype(int)
            r = max(w, h) // 2 + 6
            r = max(r, 3)
            cv2.circle(frame_display, (cx, cy), r, (0, 0, 255), 1)

        # ----------------------------
        # статичные брызги
        # ----------------------------
        if show_static:
            for sp in static_spatters[frame_idx]:
                cx, cy, w, h = sp.astype(int)
                r = max(w, h) // 2 + 6
                pts = np.array([
                    [cx, cy - r],
                    [cx - r, cy + r],
                    [cx + r, cy + r]
                ], np.int32)
                cv2.polylines(frame_display, [pts], True, (255, 255, 0), 2)

        # текст + прогресс
        text = f"Frame {frame_idx+1}/{total_frames} | FPS {fps}"
        cv2.putText(frame_display, text, (10, 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        bar_w = int((frame_idx + 1) / total_frames * frame_display.shape[1])
        cv2.rectangle(frame_display,
                      (0, frame_display.shape[0] - 12),
                      (bar_w, frame_display.shape[0]),
                      (255, 255, 255), -1)

        return frame_display

    # ---------------------------------------
    #                  MAIN LOOP
    # ---------------------------------------
    while True:
        try:
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        except cv2.error:
            break

        disp = prepare_frame_with_detections(current_frame)
        cv2.imshow(window_name, disp)

        key = cv2.waitKey(0 if paused else delay) & 0xFF

        # выход
        if key == 27:
            break
        # пауза
        elif key == ord(' '):
            paused = not paused
        # навигация
        elif key in [ord('a'), ord('A'), 81]:
            current_frame = max(0, current_frame - 1); paused = True
        elif key in [ord('d'), ord('D'), 83]:
            current_frame = min(total_frames - 1, current_frame + 1); paused = True
        elif key in [ord('w'), ord('W'), 82]:
            current_frame = min(total_frames - 1, current_frame + 10); paused = True
        elif key in [ord('s'), ord('S'), 84]:
            current_frame = max(0, current_frame - 10); paused = True
        # начало/конец
        elif key == 80: current_frame = 0; paused = True
        elif key == 87: current_frame = total_frames - 1; paused = True
        # refresh
        elif key == ord('r'):
            current_frame = 0; paused = False
        # FPS
        elif key == ord('+'):
            fps = min(60, fps + 5); delay = int(1000 / fps)
        elif key == ord('-'):
            fps = max(1, fps - 5); delay = int(1000 / fps)
        # colormap
        elif key == ord('c'):
            current_colormap_idx = (current_colormap_idx + 1) % len(available_colormaps)
            colormap = available_colormaps[current_colormap_idx]
        # normalization
        elif key == ord('n'):
            normalize = not normalize
        elif key == ord('g'):
            global_normalization = not global_normalization
            if normalize and global_normalization:
                all_values = np.concatenate([f.flatten() for f in frame_list])
                min_val = np.percentile(all_values, 1)
                max_val = np.percentile(all_values, 99)
                max_val = max(max_val, min_val + 1e-6)
        # TOGGLES (1–5)
        elif key == ord('1'): show_box = not show_box
        elif key == ord('2'): show_ellipse = not show_ellipse
        elif key == ord('3'): show_static = not show_static
        elif key == ord('4'): show_corridor = not show_corridor
        elif key == ord('5'): show_hot_stripes = not show_hot_stripes
        # автоплей
        elif not paused:
            current_frame = (current_frame + 1) % total_frames

    cv2.destroyAllWindows()
    print("Просмотр завершен")


## Детекция капель и зоны сварки

In [14]:
def detect_spatters(frame, 
                    sigma=1.8, 
                    k_size=9, 
                    threshold=11, 
                    min_size=3,
                    max_size=20):
    """Детекция брызг с настраиваемыми параметрами"""
    filtered = min_loc_LoG(frame, k_size, sigma)
    filtered = ((filtered > threshold) * 255).astype(np.uint8)
    c, _ = cv2.findContours(filtered, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [np.array([cv2.boundingRect(c[i])[0],cv2.boundingRect(c[i])[1],cv2.boundingRect(c[i])[0]+cv2.boundingRect(c[i])[2],cv2.boundingRect(c[i])[1]+cv2.boundingRect(c[i])[3]]) for i,_ in enumerate(c)]
    contours = np.array(contours)
    if not len(contours):
        return ()
    wh = contours[:, 2:4] - contours[:, :2]
    contours[:, :2] = contours[:, :2] + wh / 2
    contours[:, 2:4] = wh
    
    # ПРОСТЫЕ ОГРАНИЧЕНИЯ НА РАЗМЕР
    # Фильтруем по ширине и высоте
    width_ok = (wh[:, 0] >= min_size) & (wh[:, 0] <= max_size)
    height_ok = (wh[:, 1] >= min_size) & (wh[:, 1] <= max_size)
    size_filter = width_ok & height_ok
    
    contours = contours[size_filter]
    return contours

def detect_welding_zone(frame: np.ndarray,
                        threshold: int = 9.9,
                        min_area: int = 0.1,
                        max_fraction: float = 0.15):
    """
    Возвращает (box, ellipse).
    box = [x1, y1, x2, y2] или None
    ellipse = (center, axes, angle) или None

    max_fraction — максимальная доля кадра по ширине и высоте.
    Если зона превышает эту долю — считается, что её нет.
    """

    img = frame.astype(np.float32)
    img_norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    _, mask = cv2.threshold(img_norm, threshold, 255, cv2.THRESH_BINARY)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations=1)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None, None

    contours = [c for c in contours if cv2.contourArea(c) >= min_area]
    if not contours:
        return None, None

    largest = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest)

    # Ограничение по доле кадра
    max_w = frame.shape[1] * max_fraction
    max_h = frame.shape[0] * max_fraction
    if w > max_w or h > max_h:
        return None, None

    box = np.array([x, y, x + w, y + h])

    ellipse = None
    if len(largest) >= 5:
        ellipse = cv2.fitEllipse(largest)

    return box, ellipse

    


def detect_spatters_video(frames: np.ndarray, show_progress: bool = True) -> list:
    """
    Detect spatters in video sequence using detect_spatters function
    
    Args:
        frames: numpy array of shape (N, H, W) - video frames
        show_progress: whether to show progress bar
    
    Returns:
        List of spatter contours for each frame
    """
    all_spatters = []
    
    iterator = range(frames.shape[0])
    if show_progress:
        iterator = tqdm(iterator, desc="Detecting spatters")
    
    for i in iterator:
        spatters = detect_spatters(frames[i])
        all_spatters.append(spatters)
    
    return all_spatters


def detect_welding_zone_video(frames: np.ndarray, 
                             show_progress: bool = True,
                             max_centers_history: int = 10,
                             max_velocity_history: int = 70,
                             min_centers_for_prediction: int = 5):
    """
    Детектирует зону сварки в видео с предсказанием при отсутствии детекции
    
    Аргументы:
        frames: numpy массив формы (N, H, W) - кадры видео
        show_progress: показывать ли прогресс-бар
        max_centers_history: максимальное количество центров для хранения в истории
        max_velocity_history: максимальное количество значений скорости для хранения
        min_centers_for_prediction: минимальное количество центров необходимое для предсказания
    """
    all_boxes = []
    all_ellipses = []
    
    welding_centers_history = []
    frame_indices_history = []
    velocity_history = []
    
    iterator = range(frames.shape[0])
    if show_progress:
        iterator = tqdm(iterator, desc="Детекция зон сварки с предсказанием")

    for i in iterator:
        try:
            box, ellipse = detect_welding_zone(frames[i])
            
            if box is not None and ellipse is not None:
                # Зона сварки обнаружена
                all_boxes.append(box)
                all_ellipses.append(ellipse)
                
                # Сохраняем центр для истории
                center_x, center_y = ellipse[0]
                welding_centers_history.append((center_x, center_y))
                frame_indices_history.append(i)
                
                # Обновляем историю скоростей
                if len(welding_centers_history) >= 2:
                    last_center = welding_centers_history[-2]
                    current_center = welding_centers_history[-1]
                    frame_diff = frame_indices_history[-1] - frame_indices_history[-2]
                    
                    if frame_diff > 0:
                        velocity_x = (current_center[0] - last_center[0]) / frame_diff
                        velocity_y = (current_center[1] - last_center[1]) / frame_diff
                        velocity_history.append((velocity_x, velocity_y))
                
                # Ограничиваем размер истории
                if len(welding_centers_history) > max_centers_history:
                    welding_centers_history.pop(0)
                    frame_indices_history.pop(0)
                if len(velocity_history) > max_velocity_history:
                    velocity_history.pop(0)
                    
            else:
                # Зона сварки не обнаружена - предсказываем
                last_successful_box = None
                last_successful_ellipse = None
                
                if welding_centers_history:
                    last_successful_box = all_boxes[frame_indices_history[-1]]
                    last_successful_ellipse = all_ellipses[frame_indices_history[-1]]
                
                predicted_box, predicted_ellipse = predict_welding_zone(
                    welding_centers_history, 
                    velocity_history,
                    frame_indices_history,
                    i,
                    last_successful_box,
                    last_successful_ellipse,
                    min_centers_for_prediction
                )
                
                all_boxes.append(predicted_box)
                all_ellipses.append(predicted_ellipse)
                
        except Exception:
            # В случае ошибки предсказываем
            last_successful_box = None
            last_successful_ellipse = None
            
            if welding_centers_history:
                last_successful_box = all_boxes[frame_indices_history[-1]]
                last_successful_ellipse = all_ellipses[frame_indices_history[-1]]
            
            predicted_box, predicted_ellipse = predict_welding_zone(
                welding_centers_history, 
                velocity_history,
                frame_indices_history,
                i,
                last_successful_box,
                last_successful_ellipse,
                min_centers_for_prediction
            )
            
            all_boxes.append(predicted_box)
            all_ellipses.append(predicted_ellipse)

    return all_boxes, all_ellipses


def predict_welding_zone(centers_history, velocity_history, frame_indices_history, 
                        current_frame_idx, last_successful_box, last_successful_ellipse,
                        min_centers_for_prediction: int = 2):
    """
    Предсказывает положение зоны сварки на основе истории
    
    Аргументы:
        centers_history: список предыдущих центров [(x, y), ...]
        velocity_history: список скоростей [(vx, vy), ...]
        frame_indices_history: список номеров кадров с детекцией
        current_frame_idx: текущий номер кадра
        last_successful_box: последний успешно обнаруженный bounding box
        last_successful_ellipse: последний успешно обнаруженный эллипс
        min_centers_for_prediction: минимальное количество центров для предсказания
    """
    
    if len(centers_history) < min_centers_for_prediction or len(velocity_history) == 0:
        return last_successful_box, last_successful_ellipse
    
    # Вычисляем среднюю скорость
    avg_velocity_x = np.mean([v[0] for v in velocity_history])
    avg_velocity_y = np.mean([v[1] for v in velocity_history])
    
    # Берем последний известный центр
    last_center = centers_history[-1]
    last_frame_with_detection = frame_indices_history[-1]
    
    # Вычисляем смещение
    frames_since_last_detection = current_frame_idx - last_frame_with_detection
    displacement_x = avg_velocity_x * frames_since_last_detection
    displacement_y = avg_velocity_y * frames_since_last_detection
    
    # Новый предсказанный центр
    predicted_center_x = last_center[0] + displacement_x
    predicted_center_y = last_center[1] + displacement_y
    
    if last_successful_box is not None:
        # Предсказываем новый bounding box
        box_width = last_successful_box[2]
        box_height = last_successful_box[3]
        
        predicted_box = np.array([
            predicted_center_x - box_width / 2,
            predicted_center_y - box_height / 2,
            box_width,
            box_height
        ])
    else:
        predicted_box = None
    
    if last_successful_ellipse is not None:
        # Предсказываем новый эллипс
        (_, _), (major_axis, minor_axis), angle = last_successful_ellipse
        predicted_ellipse = ((predicted_center_x, predicted_center_y), 
                           (major_axis, minor_axis), angle)
    else:
        predicted_ellipse = None
    
    return predicted_box, predicted_ellipse

## Убираем статичные капли

In [15]:
def remove_static_spatters(spatters_list, max_shift=7, min_frames=4):
    """
    Находит статичные брызги, которые остаются в окрестности max_shift
    более чем min_frames подряд.
    Возвращает:
        cleaned_spatters — динамические
        static_spatters  — статичные
    """

    # --- ПРИВЕДЕНИЕ К НОРМАЛЬНОМУ ФОРМАТУ ---
    if isinstance(spatters_list, tuple):
        spatters_list = list(spatters_list)

    normalized = []
    for s in spatters_list:
        if isinstance(s, np.ndarray):
            normalized.append(s)
        else:
            normalized.append(np.array(s, dtype=float) if len(s) > 0 else np.empty((0, 4)))
    spatters_list = normalized

    num_frames = len(spatters_list)

    cleaned = [None] * num_frames
    static = [None] * num_frames

    for i in range(num_frames):
        cleaned[i] = []
        static[i] = []

    # --- ПОИСК СТАТИЧНЫХ ---
    for i in range(num_frames - min_frames):
        frame0 = spatters_list[i]

        for j, sp0 in enumerate(frame0):
            cx0, cy0, w0, h0 = sp0
            is_static = True

            for t in range(1, min_frames):
                frame_t = spatters_list[i + t]
                found = False

                for sp in frame_t:
                    cx, cy, w, h = sp
                    if abs(cx - cx0) <= max_shift and abs(cy - cy0) <= max_shift:
                        found = True
                        break

                if not found:
                    is_static = False
                    break

            # --- КЛАССИФИКАЦИЯ ---
            if is_static:
                # добавляем в static на эти кадры
                for t in range(min_frames):
                    static[i + t].append(sp0.tolist())

            else:
                cleaned[i].append(sp0.tolist())

    # кадры после последнего окна просто копируем
    for i in range(num_frames - min_frames, num_frames):
        cleaned[i] = spatters_list[i].copy()

    # --- СТАВИМ ПУСТЫЕ np.array, где нужно ---
    cleaned = [np.array(f) if len(f) > 0 else np.empty((0, 4)) for f in cleaned]
    static = [np.array(f) if len(f) > 0 else np.empty((0, 4)) for f in static]

    return cleaned, static


## Мертвая зона вокруг зоны сварки

In [16]:
def apply_dead_zone(spatters_list, welding_ellipses, dead_zone_radius=50):
    """
    Убирает брызги, попадающие в небольшую мертвую зону вокруг центра эллипса.
    """
    filtered = []

    for frame_idx, spatters in enumerate(spatters_list):
        if welding_ellipses[frame_idx] is None:
            # просто копируем
            filtered.append(spatters)
            continue

        # формат эллипса: ((xc, yc), (MA, ma), angle)
        (xc, yc), (MA, ma), angle = welding_ellipses[frame_idx]
        xc, yc = int(xc), int(yc)

        out = []

        for (cx, cy, w, h) in spatters:
            dist = np.hypot(cx - xc, cy - yc)

            # оставляем только те, которые ВНЕ dead zone
            if dist > dead_zone_radius:
                out.append([cx, cy, w, h])

        # приведение формата
        filtered.append(
            np.array(out) if len(out) > 0 else np.empty((0, 4))
        )

    return filtered


## Мертвый корридор за зоной сварки

In [17]:
def apply_fixed_corridor_incremental(spatters_list, welding_ellipses, corridor_width=20, lookahead=5, show_progress=True):
    """
    Фильтрует брызги внутри "накопительного коридора".
    Коридор строится каждые lookahead кадров, накопительно.
    Фильтрация брызг на кадре idx учитывает только сегменты до idx.

    Параметры:
      spatters_list     — список массивов брызг по кадрам
      welding_ellipses  — список эллипсов зоны сварки по кадрам
      corridor_width    — ширина коридора
      lookahead         — количество кадров для расчета сегмента коридора
      show_progress     — показывать прогресс-бар в консоли
    """
    filtered = []
    corridor_segments = []
    first_center = None

    iterator = tqdm(enumerate(spatters_list), total=len(spatters_list), desc="Filtering spatters") if show_progress else enumerate(spatters_list)

    for idx, spatters in iterator:
        # --- Добавляем сегмент коридора, если есть эллипс на кадре ---
        ell = welding_ellipses[idx]
        if ell is not None:
            center = np.array(ell[0], dtype=float)
            if first_center is not None:
                segment = get_corridor_segment(first_center, center, corridor_width)
                corridor_segments.append(segment)
            first_center = center

        # --- Фильтруем брызги по накопленным сегментам ---
        out = []
        for (cx, cy, w, h) in spatters:
            point = np.array([cx, cy], dtype=float)
            inside_any = False
            for seg in corridor_segments:
                a, b, _, _ = seg
                seg_vec = np.array(b) - np.array(a)
                seg_length = np.linalg.norm(seg_vec)
                if seg_length < 1e-6:
                    continue
                seg_dir = seg_vec / seg_length
                seg_perp = np.array([-seg_dir[1], seg_dir[0]])
                rel = point - np.array(a)
                proj = np.dot(rel, seg_dir)
                side = np.dot(rel, seg_perp)
                if 0 <= proj <= seg_length and abs(side) <= corridor_width / 2:
                    inside_any = True
                    break
            if not inside_any:
                out.append([cx, cy, w, h])

        filtered.append(np.array(out) if out else np.empty((0, 4)))

    return filtered


## Обнаружение горячих полос вдоль шва

In [18]:
def detect_hot_stripes_oriented(frame: np.ndarray,
                                threshold_low: int = 413,
                                threshold_high: int = 660,
                                min_area: int = 35,
                                circular_thr: float = 4.0,
                                angle_smooth: float = 0.3,
                                prev_angle: float = None):
    

    """
    Детекция вытянутых горячих полос с корректировкой угла.
    Фильтруются почти круглые эллипсы и слишком большие площади.
    Использует диапазон порогов (threshold_low, threshold_high).
    Возвращает список полос (x, y, w, h, angle) и обновленный угол шва.
    """
    if prev_angle is None:
        prev_angle = -3 * np.pi / 4

    # если верхний порог не задан, используем бесконечность
    if threshold_high is None:
        threshold_high = 65535 if frame.dtype != np.uint8 else 255

    # бинаризация по диапазону
    hot_mask = ((frame >= threshold_low) & (frame <= threshold_high)).astype(np.uint8)

    # морфология с вытянутым прямоугольным ядром
    kernel_len = 15
    ker = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 3))
    M = cv2.getRotationMatrix2D((kernel_len / 2, 1.5), degrees(prev_angle), 1.0)
    kernel_rot = cv2.warpAffine(ker, M, (kernel_len, 3))
    hot_dir = cv2.morphologyEx(hot_mask, cv2.MORPH_CLOSE, kernel_rot, iterations=1)

    # поиск связных компонент
    num, labels, stats, centroids = cv2.connectedComponentsWithStats(hot_dir)
    stripes = []
    angles_detected = []

    H, W = frame.shape[:2]
    max_allowed_area = 0.05 * (H * W)

    for i in range(1, num):
        mask_i = (labels == i).astype(np.uint8)
        area_pixels = cv2.countNonZero(mask_i)
        if area_pixels < min_area:
            continue

        pts = cv2.findNonZero(mask_i)
        if pts is not None and len(pts) >= 5:
            ellipse = cv2.fitEllipse(pts)
            (cx, cy), (MAJOR, MINOR), angle_deg = ellipse
            angle = radians(angle_deg)

            ellipse_area = pi * (MAJOR / 2) * (MINOR / 2)
            if ellipse_area > max_allowed_area:
                continue

            aspect = max(MAJOR, MINOR) / max(1e-5, min(MAJOR, MINOR))
            if aspect < circular_thr:
                continue
        else:
            x, y, w, h, _ = stats[i]
            cx, cy = x + w / 2, y + h / 2
            MAJOR, MINOR = max(w, h), min(w, h)
            angle = prev_angle

        stripes.append((cx, cy, MAJOR, MINOR, angle))
        angles_detected.append(angle)

    # обновление глобального угла
    if angles_detected:
        new_angle = np.mean(angles_detected)
        updated_angle = prev_angle + angle_smooth * (new_angle - prev_angle)
    else:
        updated_angle = prev_angle

    return stripes, updated_angle



def apply_hot_stripes_filter_oriented(cleaned_spatters, stripes):
    """
    Убирает капли, попавшие в вытянутые горячие полосы.
    cleaned_spatters: list of [x, y, w, h]
    stripes: list of (x, y, w, h, angle)
    """
    filtered = []
    for sp in cleaned_spatters:
        sx, sy = sp[0], sp[1]
        keep = True
        for x, y, w, h, angle in stripes:
            cx, cy = x + w / 2, y + h / 2
            long_side, short_side = max(w, h), min(w, h)
            cos_a, sin_a = np.cos(-angle), np.sin(-angle)
            dx, dy = sx - cx, sy - cy
            dx_r = cos_a * dx - sin_a * dy
            dy_r = sin_a * dx + cos_a * dy
            if abs(dx_r) <= long_side / 2 and abs(dy_r) <= short_side / 2:
                keep = False
                break
        if keep:
            filtered.append(sp)
    return filtered

def filter_spatters_by_hot_stripes_video_oriented(frames: np.ndarray,
                                                  cleaned_spatters_list,
                                                  threshold_low=413,
                                                  threshold_high=660,   # можно задать верхний порог
                                                  min_area=35,
                                                  circular_thr=4.0,
                                                  angle_smooth=0.3,
                                                  initial_angle=-3*np.pi/4,
                                                  show_progress=True):
    """
    Покадровая фильтрация всех капель с учетом ориентации полос.
    """
    cleaned_spatters_new = []
    hot_stripes_list = []
    current_angle = initial_angle

    iterator = range(len(frames))
    if show_progress:
        iterator = tqdm(iterator, desc="Filtering spatters by oriented hot stripes")

    for i in iterator:
        frame = frames[i]
        cleaned_spatters = cleaned_spatters_list[i]

        stripes, current_angle = detect_hot_stripes_oriented(
            frame=frame,
            threshold_low=threshold_low,
            threshold_high=threshold_high,
            min_area=min_area,
            circular_thr=circular_thr,
            angle_smooth=angle_smooth,
            prev_angle=current_angle
        )

        cleaned = apply_hot_stripes_filter_oriented(cleaned_spatters, stripes)
        cleaned_spatters_new.append(cleaned)
        hot_stripes_list.append(stripes)

    return cleaned_spatters_new, hot_stripes_list



In [19]:
def detect_and_filter_spatters_by_hot_stripes(frames: np.ndarray,
                                              cleaned_spatters_list,
                                              hot_threshold=180,
                                              min_area=20,
                                              circular_thr=4.0,
                                              angle_smooth=0.3,
                                              initial_angle=radians(45),
                                              show_progress=True):
    """
    Полная покадровая обработка:
    1) Детекция вытянутых горячих полос с учетом ориентации.
    2) Фильтрация капель внутри этих полос.
    """
    hot_stripes_list = []
    cleaned_spatters_new = []
    current_angle = initial_angle

    iterator = range(len(frames))
    if show_progress:
        iterator = tqdm(iterator, desc="Detecting and filtering hot stripes")

    for i in iterator:
        frame = frames[i]
        cleaned_spatters = cleaned_spatters_list[i]

        stripes, current_angle = detect_hot_stripes_oriented(
            frame=frame,
            hot_threshold=hot_threshold,
            min_area=min_area,
            circular_thr=circular_thr,
            angle_smooth=angle_smooth,
            prev_angle=current_angle
        )

        hot_stripes_list.append(stripes)
        cleaned = apply_hot_stripes_filter_oriented(cleaned_spatters, stripes)
        cleaned_spatters_new.append(cleaned)

    return cleaned_spatters_new, hot_stripes_list

## Вывод статистики детектирования

In [20]:
def print_detection_statistics(spatters_list, cleaned_spatters, static_spatters, welding_boxes):
    """Статистика детекции и удаления статических брызг."""

    total_frames = len(spatters_list)
    total_before = sum(len(s) for s in spatters_list)
    total_after = sum(len(s) for s in cleaned_spatters)
    total_static = sum(len(s) for s in static_spatters)

    frames_with_weld = sum(1 for box in welding_boxes if box is not None)
    frames_without_weld = total_frames - frames_with_weld

    print("\nСТАТИСТИКА ДЕТЕКЦИИ:")
    print(f"  Всего кадров: {total_frames}")
    print(f"  Кадров с зоной сварки:   {frames_with_weld} ({frames_with_weld/total_frames*100:.1f}%)")
    print(f"  Кадров без зоны сварки:  {frames_without_weld} ({frames_without_weld/total_frames*100:.1f}%)")

    print(f"\n  Всего брызг до фильтрации: {total_before}")
    print(f"  Среднее на кадр: {total_before/total_frames:.2f}")

    print(f"\n  Найдено статичных брызг: {total_static}")
    print(f"  Удалено статичных:       {total_before - total_after}")
    print(f"  Осталось динамичных:     {total_after}")


## Вызов функций

In [21]:
# # --- ФИЛЬТРАЦИЯ ИЗОБРАЖЕНИЯ ДЛЯ ПОИСКА МЕЛКИХ БРЫЗГ ---
# filtered_video = np.load(output_path)

In [22]:
# # --- ДЕТЕКЦИЯ ---
# spatters_list = detect_spatters_video(filtered_video)
# welding_boxes, welding_ellipses = detect_welding_zone_video(frames)

In [23]:
# # --- ДОБАВЛЕНИЕ МЕРТВОЙ ЗОНЫ ВОКРУГ СВАРОЧНОЙ ВАННЫ ---
# cleaned_spatters = apply_dead_zone(spatters_list, welding_ellipses, dead_zone_radius=50)

In [24]:
# # --- ДОБАВЛЕНИЕ МЕРТВОГО КОРИДОРА ---
# cleaned_spatters = apply_fixed_corridor_incremental(cleaned_spatters, welding_ellipses, corridor_width=35, lookahead=25)

In [25]:
# # --- ДОБАВЛЕНИЕ ГОРЯЧИХ ПОЛОС ---
# cleaned_spatters, hot_stripes_list = filter_spatters_by_hot_stripes_video_oriented(
#                                             frames=frames,
#                                             cleaned_spatters_list=cleaned_spatters,
#                                             threshold_low=500,
#                                             threshold_high=1200,  
#                                             min_area=35,
#                                             circular_thr=4.0,
#                                             angle_smooth=0.3,
#                                             initial_angle=-3*np.pi/4,
#                                             show_progress=True
# )

In [26]:
# # --- ФИЛЬТРАЦИЯ СТАТИЧЕСКИХ БРЫЗГ ---
# cleaned_spatters, static_spatters = remove_static_spatters(cleaned_spatters,
#                                                            max_shift=5,
#                                                            min_frames=10)

## Смотрим видеВо

In [27]:
# # --- СТАТИСТИКА ---
# print_detection_statistics(
#     spatters_list,
#     cleaned_spatters,
#     static_spatters,
#     welding_boxes
# )

# # --- ВИЗУАЛИЗАЦИЯ ---
# show_video_with_detections(
#     frames,
#     cleaned_spatters,
#     static_spatters,
#     welding_boxes,
#     welding_ellipses,
#     hot_stripes_list=hot_stripes_list,
#     fps=10
# )

## Вытаскиваем данные и формируем датафрейм

### Количество капель

In [28]:
def create_spatters_count_dataframe(cleaned_spatters, thermogram_id=thermogram_id):
    """
    Создает датафрейм с количеством брызг на каждый кадр
    
    Args:
        cleaned_spatters: список массивов брызг по кадрам
        thermogram_id: номер термограммы (опционально)
    
    Returns:
        DataFrame с колонками ['frame_number', 'spatters_count']
    """
    
    # Создаем список с количеством брызг на каждый кадр
    spatters_count = [len(frame_spatters) for frame_spatters in cleaned_spatters]
    
    # Создаем датафрейм
    df = pd.DataFrame({
        'frame_number': range(1, len(cleaned_spatters) + 1),
        'spatters_count_del': spatters_count #возьмем из трекера
    })
    
    # Добавляем ID термограммы если указан
    if thermogram_id is not None:
        df['thermogram_id'] = thermogram_id
    
    return df

### Медианная температура

In [29]:
def calculate_spatters_median_temperature(cleaned_spatters, frames, threshold_temp=420, thermogram_id=thermogram_id):
    """
    Рассчитывает медианную температуру брызг для каждого кадра с применением порога температуры.

    Аргументы:
        cleaned_spatters: список массивов брызг по кадрам в формате [cx, cy, w, h]
        frames: исходные кадры с температурными данными (numpy arrays)
        threshold_temp: порог температуры - пиксели ниже этого значения игнорируются при расчетах
        thermogram_id: идентификатор термограммы для группировки результатов

    Возвращает:
        DataFrame со следующими колонками:
        - 'frame_number': номер кадра (начинается с 1)
        - 'spatters_count': общее количество обнаруженных брызг на кадре
        - 'spatters_above_threshold': количество брызг, имеющих хотя бы один пиксель выше порога
        - 'median_spatter_temperature': медианная температура брызг (рассчитанная только по пикселям выше порога)
        - 'mean_spatter_temperature': средняя температура брызг (рассчитанная только по пикселям выше порога)
        - 'max_spatter_temperature': максимальная температура среди всех брызг (по пикселям выше порога)
        - 'total_pixels_above_threshold': общее количество пикселей во всех брызгах, превышающих порог
        - 'percentage_above_threshold': процент пикселей выше порога от общего количества пикселей в брызгах
        - 'threshold_used': значение использованного порога температуры
        - 'thermogram_id': идентификатор термограммы (если передан)

    Примечания:
        - Для кадров без брызг все температурные показатели устанавливаются в np.nan
        - Расчет температур производится только для пикселей, превышающих заданный порог
        - Каждая брызга учитывается в статистике только если содержит хотя бы один пиксель выше порога
    """
    results = []
    
    for frame_idx in range(len(cleaned_spatters)):
        frame_spatters = cleaned_spatters[frame_idx]
        frame_data = frames[frame_idx]
        
        if len(frame_spatters) == 0:
            # Если брызг нет, записываем NaN
            frame_result = {
                'frame_number': frame_idx + 1,
                'spatters_above_threshold': 0,
                'spatters_median_temperature': np.nan,
                'spatters_mean_temperature': np.nan,
                'spatters_max_temperature': np.nan,
                'spatters_total_pixels_above_threshold': 0,
                'spatters_percentage_above_threshold': 0.0
            }
        else:
            spatter_mean_temperatures = []
            spatters_above_threshold = 0
            total_pixels_above = 0
            total_pixels_all = 0
            
            for spatter in frame_spatters:
                cx, cy, w, h = spatter.astype(int)
                
                # Вычисляем bounding box брызги
                x_start = max(0, cx - w//2)
                x_end = min(frame_data.shape[1], cx + w//2 + 1)
                y_start = max(0, cy - h//2)
                y_end = min(frame_data.shape[0], cy + h//2 + 1)
                
                # Извлекаем пиксели брызги
                spatter_pixels = frame_data[y_start:y_end, x_start:x_end]
                
                if spatter_pixels.size > 0:
                    # ПРИМЕНЯЕМ ПОРОГ: оставляем только пиксели выше threshold_temp
                    above_threshold_pixels = spatter_pixels[spatter_pixels >= threshold_temp]
                    total_pixels_all += spatter_pixels.size
                    total_pixels_above += above_threshold_pixels.size
                    
                    if above_threshold_pixels.size > 0:
                        # Средняя температура для одной брызги (только выше порога)
                        mean_temp = np.mean(above_threshold_pixels)
                        spatter_mean_temperatures.append(mean_temp)
                        spatters_above_threshold += 1
            
            # Рассчитываем статистики
            if spatter_mean_temperatures:
                median_temp = np.median(spatter_mean_temperatures)
                mean_temp = np.mean(spatter_mean_temperatures)
                max_temp = np.max(spatter_mean_temperatures)
                percentage_above = (total_pixels_above / total_pixels_all * 100) if total_pixels_all > 0 else 0
            else:
                median_temp = np.nan
                mean_temp = np.nan
                max_temp = np.nan
                percentage_above = 0.0
            
            frame_result = {
                'frame_number': frame_idx + 1,
                'spatters_above_threshold': spatters_above_threshold,
                'spatters_median_temperature': median_temp,
                'spatters_mean_temperature': mean_temp,
                'spatters_max_temperature': max_temp,
                'spatters_total_pixels_above_threshold': total_pixels_above,
                'spatters_percentage_above_threshold': percentage_above,
            }
        
        results.append(frame_result)
    
    df = pd.DataFrame(results)

    return df

## Функция настройки порога для подсчета температуры брызг

In [30]:
def visualize_spatters_threshold(frame_idx, cleaned_spatters, frames, threshold_temp=420, max_spatters_to_show=10):
    """
    Визуализирует брызги с применением порога температуры для настройки
    
    Аргументы:
        frame_idx: номер кадра для визуализации
        cleaned_spatters: список массивов брызг
        frames: кадры с температурными данными
        threshold_temp: порог температуры
        max_spatters_to_show: максимальное количество брызг для отображения
    """
    
    frame_spatters = cleaned_spatters[frame_idx]
    frame_data = frames[frame_idx]
    
    if len(frame_spatters) == 0:
        print(f"На кадре {frame_idx} брызг не обнаружено")
        return
    
    print(f"=== АНАЛИЗ БРЫЗГ НА КАДРЕ {frame_idx} ===")
    print(f"Всего брызг: {len(frame_spatters)}")
    print(f"Порог температуры: {threshold_temp}")
    print("-" * 50)
    
    # Статистика по всем брызгам
    all_spatter_stats = []
    
    for i, spatter in enumerate(frame_spatters[:max_spatters_to_show]):
        cx, cy, w, h = spatter.astype(int)
        
        # Вычисляем bounding box
        x_start = max(0, cx - w//2)
        x_end = min(frame_data.shape[1], cx + w//2 + 1)
        y_start = max(0, cy - h//2)
        y_end = min(frame_data.shape[0], cy + h//2 + 1)
        
        # Извлекаем пиксели
        spatter_pixels = frame_data[y_start:y_end, x_start:x_end]
        
        if spatter_pixels.size > 0:
            # Пиксели выше и ниже порога
            above_threshold = spatter_pixels[spatter_pixels >= threshold_temp]
            below_threshold = spatter_pixels[spatter_pixels < threshold_temp]
            
            spatter_stat = {
                'spatter_id': i + 1,
                'center': (cx, cy),
                'size': (w, h),
                'total_pixels': spatter_pixels.size,
                'pixels_above_threshold': above_threshold.size,
                'pixels_below_threshold': below_threshold.size,
                'percentage_above': (above_threshold.size / spatter_pixels.size * 100) if spatter_pixels.size > 0 else 0,
                'mean_temp_above': np.mean(above_threshold) if above_threshold.size > 0 else np.nan,
                'max_temp_above': np.max(above_threshold) if above_threshold.size > 0 else np.nan,
                'mean_temp_all': np.mean(spatter_pixels),
                'max_temp_all': np.max(spatter_pixels)
            }
            
            all_spatter_stats.append(spatter_stat)
            
            print(f"Брызга {i+1}:")
            print(f"  Центр: ({cx}, {cy}), Размер: {w}x{h}")
            print(f"  Пикселей: {spatter_pixels.size}")
            print(f"  Выше порога: {above_threshold.size} ({spatter_stat['percentage_above']:.1f}%)")
            if above_threshold.size > 0:
                print(f"  Средняя темп. выше порога: {spatter_stat['mean_temp_above']:.2f}")
                print(f"  Макс. темп. выше порога: {spatter_stat['max_temp_above']:.2f}")
            print(f"  Средняя темп. всех пикселей: {spatter_stat['mean_temp_all']:.2f}")
            print(f"  Макс. темп. всех пикселей: {spatter_stat['max_temp_all']:.2f}")
            print()
    
    # Общая статистика
    if all_spatter_stats:
        total_spatters_above = sum(1 for s in all_spatter_stats if s['pixels_above_threshold'] > 0)
        avg_percentage_above = np.mean([s['percentage_above'] for s in all_spatter_stats])
        
        print("ОБЩАЯ СТАТИСТИКА:")
        print(f"Брызг выше порога: {total_spatters_above}/{len(all_spatter_stats)}")
        print(f"Средний % пикселей выше порога: {avg_percentage_above:.1f}%")
    
    # ВИЗУАЛИЗАЦИЯ
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Исходный кадр с bounding boxes
    axes[0, 0].imshow(frame_data, cmap='hot')
    for i, spatter in enumerate(frame_spatters[:max_spatters_to_show]):
        cx, cy, w, h = spatter.astype(int)
        rect = plt.Rectangle((cx - w//2, cy - h//2), w, h, 
                           fill=False, color='red', linewidth=1.5)
        axes[0, 0].add_patch(rect)
        axes[0, 0].text(cx, cy - h//2 - 5, f'{i+1}', color='white', 
                       fontsize=8, ha='center', va='bottom')
    axes[0, 0].set_title(f'Кадр {frame_idx} - Bounding Boxes брызг')
    axes[0, 0].set_xlabel('X')
    axes[0, 0].set_ylabel('Y')
    
    # 2. Кадр с применением порога (пиксели ниже порога - черные)
    display_frame = frame_data.copy()
    for spatter in frame_spatters:
        cx, cy, w, h = spatter.astype(int)
        x_start = max(0, cx - w//2)
        x_end = min(frame_data.shape[1], cx + w//2 + 1)
        y_start = max(0, cy - h//2)
        y_end = min(frame_data.shape[0], cy + h//2 + 1)
        
        # Обнуляем пиксели ниже порога в bounding boxes
        spatter_region = display_frame[y_start:y_end, x_start:x_end]
        below_threshold_mask = spatter_region < threshold_temp
        spatter_region[below_threshold_mask] = 0
    
    axes[0, 1].imshow(display_frame, cmap='hot')
    axes[0, 1].set_title(f'Кадр {frame_idx} - Пиксели выше {threshold_temp}')
    axes[0, 1].set_xlabel('X')
    axes[0, 1].set_ylabel('Y')
    
    # 3. Гистограмма температур всех пикселей брызг
    all_spatter_pixels = []
    for spatter in frame_spatters:
        cx, cy, w, h = spatter.astype(int)
        x_start = max(0, cx - w//2)
        x_end = min(frame_data.shape[1], cx + w//2 + 1)
        y_start = max(0, cy - h//2)
        y_end = min(frame_data.shape[0], cy + h//2 + 1)
        
        spatter_pixels = frame_data[y_start:y_end, x_start:x_end]
        all_spatter_pixels.extend(spatter_pixels.flatten())
    
    if all_spatter_pixels:
        axes[1, 0].hist(all_spatter_pixels, bins=50, alpha=0.7, color='blue', edgecolor='black')
        axes[1, 0].axvline(x=threshold_temp, color='red', linestyle='--', 
                          linewidth=2, label=f'Порог: {threshold_temp}')
        axes[1, 0].set_xlabel('Температура')
        axes[1, 0].set_ylabel('Количество пикселей')
        axes[1, 0].set_title('Распределение температур всех пикселей брызг')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Процент пикселей выше порога по брызгам
    if all_spatter_stats:
        spatter_ids = [s['spatter_id'] for s in all_spatter_stats]
        percentages_above = [s['percentage_above'] for s in all_spatter_stats]
        
        bars = axes[1, 1].bar(spatter_ids, percentages_above, color='lightcoral', alpha=0.7)
        
        # Подкрашиваем брызги с высоким процентом
        for i, (bar, percentage) in enumerate(zip(bars, percentages_above)):
            if percentage > 50:
                bar.set_color('red')
            elif percentage == 0:
                bar.set_color('gray')
        
        axes[1, 1].set_xlabel('Номер брызги')
        axes[1, 1].set_ylabel('Процент пикселей выше порога (%)')
        axes[1, 1].set_title('Процент "горячих" пикселей по брызгам')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return all_spatter_stats

# # 1. Визуализация для настройки порога
# spatter_stats = visualize_spatters_threshold(
#     frame_idx=749,
#     cleaned_spatters=cleaned_spatters,
#     frames=frames,
#     threshold_temp=420,  # МЕНЯЙТЕ ЭТО ЗНАЧЕНИЕ ДЛЯ НАСТРОЙКИ
#     max_spatters_to_show=15
# )

In [31]:
# # Использование:
# df_n_spatters = create_spatters_count_dataframe(cleaned_spatters)

# # Использование:
# df_spatters_temperature = calculate_spatters_median_temperature(
#     cleaned_spatters=cleaned_spatters,
#     frames=frames,
#     threshold_temp=420,  # Используем настроенный порог
# )

In [32]:
# df_n_spatters

In [33]:
# df_spatters_temperature 

### Температура в зоне сварки

In [70]:
def calculate_welding_zone_temperature_threshold(welding_ellipses, frames, threshold_temp=400, radius=50, thermogram_id=None):
    """
    Рассчитывает среднюю температуру в зоне сварки с применением порога температуры
    
    Аргументы:
        welding_ellipses: список эллипсов зоны сварки по кадрам
        frames: исходные кадры с температурными данными
        threshold_temp: порог температуры (пиксели ниже этого значения игнорируются)
        radius: радиус зоны вокруг центра эллипса
        thermogram_id: номер термограммы
    
    Возвращает:
        DataFrame со средней и максимальной температурой зоны сварки по кадрам после применения порога
        и размер зона сварки выше порога в пикселях
    """
    
    results = []
    
    for frame_idx in range(len(welding_ellipses)):
        frame_ellipse = welding_ellipses[frame_idx]
        frame_data = frames[frame_idx]
        
        if frame_ellipse is None:
            # Если зоны сварки нет, записываем NaN
            frame_result = {
                'frame_number': frame_idx + 1,
                'welding_zone_detected': False,
                'welding_zone_mean_temperature': np.nan,
                'welding_zone_max_temperature': np.nan,
                'welding_zone_size': 0
            }
        else:
            # Извлекаем центр эллипса
            center_x, center_y = frame_ellipse[0]
            center_x, center_y = int(center_x), int(center_y)
            
            # Определяем границы круговой зоны
            x_start = max(0, center_x - radius)
            x_end = min(frame_data.shape[1], center_x + radius + 1)
            y_start = max(0, center_y - radius)
            y_end = min(frame_data.shape[0], center_y + radius + 1)
            
            # Извлекаем пиксели в круговой зоне
            welding_zone_pixels = frame_data[y_start:y_end, x_start:x_end]
            
            # Создаем маску для круговой области
            y_coords, x_coords = np.ogrid[y_start:y_end, x_start:x_end]
            distance_mask = ((x_coords - center_x)**2 + (y_coords - center_y)**2) <= radius**2
            
            # Применяем круговую маску и порог температуры
            masked_pixels = welding_zone_pixels[distance_mask]
            above_threshold_pixels = masked_pixels[masked_pixels >= threshold_temp]
            
            if above_threshold_pixels.size > 0:
                # Средняя температура пикселей выше порога
                mean_temp = np.mean(above_threshold_pixels)
                max_temp = np.max(above_threshold_pixels)
                pixels_count = above_threshold_pixels.size
            else:
                mean_temp = np.nan
                max_temp = np.nan
                pixels_count = 0
            
            frame_result = {
                'frame_number': frame_idx + 1,
                'welding_zone_detected': True,
                'welding_zone_mean_temperature': mean_temp,
                'welding_zone_max_temperature': max_temp,
                'welding_zone_size': pixels_count
            }
        
        results.append(frame_result)
    
    df = pd.DataFrame(results)
    
    # Добавляем колонку test_zone
    df['test_zone'] = None
    
    # Находим кадры где welding_zone_size > 10
    valid_frames = df[df['welding_zone_size'] > 10]
    
    if len(valid_frames) > 0:
        # Находим первый и последний кадры с welding_zone_size > 10
        first_frame = valid_frames['frame_number'].min()
        last_frame = valid_frames['frame_number'].max()
        
        # Вычисляем общее количество кадров в интервале
        total_frames_in_interval = last_frame - first_frame + 1
        
        # Делим на 4 равные части
        quarter_size = total_frames_in_interval // 4
        
        # Назначаем метки A, B, C, D для каждой четверти
        for i in range(4):
            start_frame = first_frame + i * quarter_size
            end_frame = first_frame + (i + 1) * quarter_size - 1 if i < 3 else last_frame
            
            # Определяем метку для текущей четверти
            zone_label = ['A', 'B', 'C', 'D'][i]
            
            # Записываем метку в test_zone для кадров этой четверти
            mask = (df['frame_number'] >= start_frame) & (df['frame_number'] <= end_frame)
            df.loc[mask, 'test_zone'] = zone_label
    
    return df

In [35]:
# # Использование:
# df_welding_temperature = calculate_welding_zone_temperature_threshold(
#     welding_ellipses=welding_ellipses, 
#     frames=frames, 
#     threshold_temp=1000,  # можно менять порог
#     radius=50,           # можно менять радиус
#     thermogram_id=thermogram_id
# )

In [36]:
# df_welding_temperature

### Функция настройки трэшхолда

In [37]:
def visualize_welding_zone_threshold(frame_idx, welding_ellipses, frames, threshold_temp=400, radius=50):
    """
    Визуализирует зону сварки с применением порога температуры для настройки
    
    Аргументы:
        frame_idx: номер кадра для визуализации
        welding_ellipses: список эллипсов зоны сварки
        frames: кадры с температурными данными
        threshold_temp: порог температуры
        radius: радиус зоны
    """
    
    frame_ellipse = welding_ellipses[frame_idx]
    frame_data = frames[frame_idx]
    
    if frame_ellipse is None:
        print(f"На кадре {frame_idx} зона сварки не обнаружена")
        return
    
    # Извлекаем центр эллипса
    center_x, center_y = frame_ellipse[0]
    center_x, center_y = int(center_x), int(center_y)
    
    # Определяем границы круговой зоны
    x_start = max(0, center_x - radius)
    x_end = min(frame_data.shape[1], center_x + radius + 1)
    y_start = max(0, center_y - radius)
    y_end = min(frame_data.shape[0], center_y + radius + 1)
    
    # Извлекаем пиксели в круговой зоне
    welding_zone_pixels = frame_data[y_start:y_end, x_start:x_end]
    
    # Создаем маску для круговой области
    y_coords, x_coords = np.ogrid[y_start:y_end, x_start:x_end]
    distance_mask = ((x_coords - center_x)**2 + (y_coords - center_y)**2) <= radius**2
    
    # Применяем круговую маску и порог температуры
    masked_pixels = welding_zone_pixels[distance_mask]
    above_threshold_pixels = masked_pixels[masked_pixels >= threshold_temp]
    below_threshold_pixels = masked_pixels[masked_pixels < threshold_temp]
    
    # Статистика
    total_pixels = masked_pixels.size
    above_count = above_threshold_pixels.size
    below_count = below_threshold_pixels.size
    
    print(f"=== АНАЛИЗ КАДРА {frame_idx} ===")
    print(f"Центр зоны: ({center_x}, {center_y})")
    print(f"Радиус зоны: {radius} пикселей")
    print(f"Порог температуры: {threshold_temp}")
    print(f"Всего пикселей в зоне: {total_pixels}")
    print(f"Пикселей выше порога: {above_count} ({above_count/total_pixels*100:.1f}%)")
    print(f"Пикселей ниже порога: {below_count} ({below_count/total_pixels*100:.1f}%)")
    
    if above_count > 0:
        print(f"Температура выше порога:")
        print(f"  Средняя: {np.mean(above_threshold_pixels):.2f}")
        print(f"  Максимальная: {np.max(above_threshold_pixels):.2f}")
        print(f"  Минимальная: {np.min(above_threshold_pixels):.2f}")
        print(f"  Медианная: {np.median(above_threshold_pixels):.2f}")
    
    print(f"Общая статистика зоны:")
    print(f"  Средняя температура всех пикселей: {np.mean(masked_pixels):.2f}")
    print(f"  Максимальная температура: {np.max(masked_pixels):.2f}")
    print(f"  Минимальная температура: {np.min(masked_pixels):.2f}")
    
    # Визуализация
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 1. Исходный кадр с зоной (все что ниже порога - черным)
    display_frame = frame_data.copy()
    
    # Создаем маску для всей зоны сварки
    y_indices, x_indices = np.indices(frame_data.shape)
    zone_mask = ((x_indices - center_x)**2 + (y_indices - center_y)**2) <= radius**2
    
    # Все что ниже порога в зоне сварки делаем черным
    below_threshold_mask = zone_mask & (frame_data < threshold_temp)
    display_frame[below_threshold_mask] = 0  # черный цвет
    
    axes[0].imshow(display_frame, cmap='hot')
    axes[0].add_patch(plt.Circle((center_x, center_y), radius, fill=False, color='red', linewidth=2))
    axes[0].plot(center_x, center_y, 'rx', markersize=10)
    axes[0].set_title(f'Кадр {frame_idx} - Зона сварки\n(ниже {threshold_temp} - черный)')
    axes[0].set_xlabel('X')
    axes[0].set_ylabel('Y')
    
    # 2. Гистограмма температур в зоне
    axes[1].hist(masked_pixels, bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[1].axvline(x=threshold_temp, color='red', linestyle='--', linewidth=2, label=f'Порог: {threshold_temp}')
    axes[1].set_xlabel('Температура')
    axes[1].set_ylabel('Количество пикселей')
    axes[1].set_title('Распределение температур в зоне')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # 3. Сравнение пикселей выше/ниже порога
    temperatures_above = above_threshold_pixels if above_count > 0 else np.array([])
    temperatures_below = below_threshold_pixels if below_count > 0 else np.array([])
    
    if above_count > 0 or below_count > 0:
        data_to_plot = []
        labels = []
        if below_count > 0:
            data_to_plot.append(temperatures_below)
            labels.append(f'Ниже порога\n({below_count} пикс.)')
        if above_count > 0:
            data_to_plot.append(temperatures_above)
            labels.append(f'Выше порога\n({above_count} пикс.)')
        
        axes[2].boxplot(data_to_plot, labels=labels)
        axes[2].set_ylabel('Температура')
        axes[2].set_title('Сравнение температур по порогу')
        axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return above_threshold_pixels

# # Использование для настройки порога:
# test_frame_idx = 1000  # можно изменить на любой кадр
# above_threshold_pixels = visualize_welding_zone_threshold(
#     frame_idx=test_frame_idx,
#     welding_ellipses=welding_ellipses,  # или welding_ellipses в зависимости от вашей переменной
#     frames=frames,
#     threshold_temp=1000,  # МЕНЯЙТЕ ЭТО ЗНАЧЕНИЕ ДЛЯ НАСТРОЙКИ
#     radius=50            # МЕНЯЙТЕ ЭТО ЗНАЧЕНИЕ ДЛЯ НАСТРОЙКИ
# )

# print("\n")

In [38]:
import importlib
import pickle
importlib.invalidate_caches()

from spatter_tracker import SpatterTracker, track_spatters_with_ids

In [39]:
# Посмотреть размерность массива кадров
print(f"Размерность frames: {frames.shape}")

# Если frames 3D массив (кадры, высота, ширина)
if frames.ndim == 3:
    print(f"Количество кадров: {frames.shape[0]}")
    print(f"Высота кадра: {frames.shape[1]}")
    print(f"Ширина кадра: {frames.shape[2]}")
    
# Если frames 4D массив (например, с цветовыми каналами)
elif frames.ndim == 4:
    print(f"Количество кадров: {frames.shape[0]}")
    print(f"Высота кадра: {frames.shape[1]}")
    print(f"Ширина кадра: {frames.shape[2]}")
    print(f"Количество каналов: {frames.shape[3]}")


Размерность frames: (1500, 256, 320)
Количество кадров: 1500
Высота кадра: 256
Ширина кадра: 320


In [40]:
# # Получаем брызги с ID и статистику
# cleaned_spatters_id, df_stats = track_spatters_with_ids(
#     cleaned_spatters=cleaned_spatters,
#     frame_width=320,
#     frame_height=256,
#     min_movement_distance=5,   # удаляем брызги которые движутся <5px за 3 кадра
#     min_movement_frames=3
# )

In [41]:
# df_stats

In [42]:
def show_video_with_tracking(frames: np.ndarray,
                           cleaned_spatters_id: list,
                           welding_boxes: list = None,
                           welding_ellipses: list = None,
                           fps: int = 25,
                           window_name: str = "Spatter Tracking",
                           colormap: int = cv2.COLORMAP_JET,
                           normalize: bool = True,
                           global_normalization: bool = True,
                           trail_length: int = 10,
                           show_trails: bool = True):
    """
    Визуализация трекинга брызг с цветами по ID и траекториями
    
    Args:
        frames: numpy array of shape (N, H, W) - video frames
        cleaned_spatters_id: список массивов [x, y, w, h, id] для каждого кадра
        welding_boxes: список боксов зоны сварки (опционально)
        welding_ellipses: список эллипсов зоны сварки (опционально)
        fps: frames per second
        window_name: имя окна
        colormap: цветовая карта OpenCV
        normalize: нормализация температуры
        global_normalization: глобальная или локальная нормализация
        trail_length: длина траектории (количество предыдущих кадров)
        show_trails: показывать траектории движения
    """
    
    if frames.ndim != 3:
        raise ValueError("Input array must be 3D")

    frame_list = [frames[i] for i in range(frames.shape[0])]
    total_frames = len(frame_list)
    delay = int(1000 / fps)
    current_frame = 0
    paused = False

    available_colormaps = [
        cv2.COLORMAP_JET, cv2.COLORMAP_HOT, cv2.COLORMAP_COOL,
        cv2.COLORMAP_TURBO, cv2.COLORMAP_VIRIDIS
    ]
    current_colormap_idx = available_colormaps.index(colormap) if colormap in available_colormaps else 0
    colormap = available_colormaps[current_colormap_idx]

    if normalize and global_normalization:
        all_values = np.concatenate([f.flatten() for f in frame_list])
        min_val = np.percentile(all_values, 1)
        max_val = np.percentile(all_values, 99)
        max_val = max(max_val, min_val + 1e-6)
    else:
        min_val = max_val = None

    # Словарь для хранения истории позиций по ID
    position_history = {}
    
    # Генерация цветовой палитры для ID
    def get_color_for_id(spatter_id):
        np.random.seed(spatter_id % 1000)  # для воспроизводимости
        return tuple(np.random.randint(0, 255, 3).tolist())

    # ----------------------------
    # FLAGS TOGGLERS (из основной функции)
    # ----------------------------
    show_box = False
    show_ellipse = True
    show_corridor = False

    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 1100, 900)

    def prepare_frame_with_tracking(frame_idx: int) -> np.ndarray:
        frame = frame_list[frame_idx]

        # Нормализация
        if normalize:
            if global_normalization:
                frame_clip = np.clip(frame, min_val, max_val)
                frame_norm = ((frame_clip - min_val) / (max_val - min_val) * 255).astype(np.uint8)
            else:
                f_min = np.percentile(frame, 1)
                f_max = np.percentile(frame, 99)
                f_max = max(f_max, f_min + 1e-6)
                frame_clip = np.clip(frame, f_min, f_max)
                frame_norm = ((frame_clip - f_min) / (f_max - f_min) * 255).astype(np.uint8)
        else:
            frame_norm = np.clip(frame, 0, 255).astype(np.uint8)

        frame_display = cv2.applyColorMap(frame_norm, colormap)

        # Обновляем историю позиций
        current_spatters = cleaned_spatters_id[frame_idx]
        
        # Удаляем старые записи из истории
        current_ids = set()
        if len(current_spatters) > 0:
            for spatter in current_spatters:
                spatter_id = int(spatter[4])
                current_ids.add(spatter_id)
                x, y = int(spatter[0]), int(spatter[1])
                
                # Добавляем текущую позицию в историю
                if spatter_id not in position_history:
                    position_history[spatter_id] = []
                position_history[spatter_id].append((frame_idx, x, y))
                
                # Ограничиваем длину истории
                if len(position_history[spatter_id]) > trail_length:
                    position_history[spatter_id] = position_history[spatter_id][-trail_length:]
        
        # Очищаем историю для ID которых больше нет
        ids_to_remove = []
        for spatter_id in position_history:
            if spatter_id not in current_ids:
                # Проверяем, не устарела ли история
                if position_history[spatter_id]:
                    last_frame, _, _ = position_history[spatter_id][-1]
                    if frame_idx - last_frame > trail_length:
                        ids_to_remove.append(spatter_id)
        
        for spatter_id in ids_to_remove:
            del position_history[spatter_id]

        # Рисуем траектории
        if show_trails:
            for spatter_id, history in position_history.items():
                if len(history) > 1:
                    color = get_color_for_id(spatter_id)
                    # Рисуем линии между точками истории
                    points = []
                    for hist_frame_idx, x, y in history:
                        # Пропускаем точки из будущих кадров (на всякий случай)
                        if hist_frame_idx <= frame_idx:
                            points.append((x, y))
                    
                    if len(points) >= 2:
                        # Рисуем плавную траекторию
                        for i in range(1, len(points)):
                            alpha = i / len(points)  # прозрачность для старых точек
                            thickness = max(1, int(3 * alpha))
                            cv2.line(frame_display, points[i-1], points[i], color, thickness)
                        
                        # Рисуем точки на траектории
                        for i, (x, y) in enumerate(points):
                            alpha = i / len(points)
                            radius = max(1, int(3 * alpha))
                            cv2.circle(frame_display, (x, y), radius, color, -1)

        # Рисуем текущие брызги
        if len(current_spatters) > 0:
            for spatter in current_spatters:
                x, y, w, h, spatter_id = spatter
                x, y = int(x), int(y)
                spatter_id = int(spatter_id)
                
                color = get_color_for_id(spatter_id)
                
                # Рисуем круг для брызги
                radius = max(3, int(max(w, h) // 2 + 2))
                cv2.circle(frame_display, (x, y), radius, color, 2)
                
                # Рисуем крестик в центре
                cross_size = radius + 2
                cv2.line(frame_display, (x-cross_size, y), (x+cross_size, y), color, 1)
                cv2.line(frame_display, (x, y-cross_size), (x, y+cross_size), color, 1)

        # Рисуем зону сварки (если есть)
        if welding_ellipses is not None and welding_ellipses[frame_idx] is not None and show_ellipse:
            try:
                cv2.ellipse(frame_display, welding_ellipses[frame_idx], (0, 255, 0), 2)
            except:
                pass

        if welding_boxes is not None and welding_boxes[frame_idx] is not None and show_box:
            try:
                cx, cy, w, h = map(int, welding_boxes[frame_idx])
                x1, y1 = cx - w // 2, cy - h // 2
                x2, y2 = cx + w // 2, cy + h // 2
                cv2.rectangle(frame_display, (x1, y1), (x2, y2), (0, 255, 255), 1)
            except:
                pass

        # Только номер кадра и прогресс-бар (без статистики)
        text = f"Frame {frame_idx+1}/{total_frames} | FPS {fps}"
        cv2.putText(frame_display, text, (10, 25),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Прогресс-бар
        bar_w = int((frame_idx + 1) / total_frames * frame_display.shape[1])
        cv2.rectangle(frame_display,
                     (0, frame_display.shape[0] - 12),
                     (bar_w, frame_display.shape[0]),
                     (255, 255, 255), -1)

        return frame_display

    # Управление
    while True:
        try:
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        except cv2.error:
            break

        disp = prepare_frame_with_tracking(current_frame)
        cv2.imshow(window_name, disp)

        key = cv2.waitKey(0 if paused else delay) & 0xFF

        # Управление (аналогично оригинальной функции)
        if key == 27: break  # ESC
        elif key == ord(' '): paused = not paused
        elif key in [ord('a'), ord('A'), 81]: current_frame = max(0, current_frame - 1); paused = True
        elif key in [ord('d'), ord('D'), 83]: current_frame = min(total_frames - 1, current_frame + 1); paused = True
        elif key in [ord('w'), ord('W'), 82]: current_frame = min(total_frames - 1, current_frame + 10); paused = True
        elif key in [ord('s'), ord('S'), 84]: current_frame = max(0, current_frame - 10); paused = True
        elif key == 80: current_frame = 0; paused = True
        elif key == 87: current_frame = total_frames - 1; paused = True
        elif key == ord('r'): current_frame = 0; paused = False
        elif key == ord('+'): fps = min(60, fps + 5); delay = int(1000 / fps)
        elif key == ord('-'): fps = max(1, fps - 5); delay = int(1000 / fps)
        elif key == ord('c'): current_colormap_idx = (current_colormap_idx + 1) % len(available_colormaps); colormap = available_colormaps[current_colormap_idx]
        elif key == ord('n'): normalize = not normalize
        elif key == ord('g'): global_normalization = not global_normalization
        # Флаги из основной функции
        elif key == ord('1'): show_box = not show_box
        elif key == ord('2'): show_ellipse = not show_ellipse
        elif key == ord('4'): show_corridor = not show_corridor
        # Управление трекингом
        elif key == ord('t'): show_trails = not show_trails  # вкл/выкл траектории
        elif key == ord(']'): trail_length = min(50, trail_length + 1)  # увеличить длину траектории
        elif key == ord('['): trail_length = max(1, trail_length - 1)  # уменьшить длину траектории
        # Автоплей
        elif not paused: current_frame = (current_frame + 1) % total_frames

    cv2.destroyAllWindows()
    print("Просмотр трекинга завершен")

## Проверка работы трекера

In [43]:
# show_video_with_tracking(
#     frames=frames,
#     cleaned_spatters_id=cleaned_spatters_id,
#     welding_boxes=welding_boxes,
#     welding_ellipses=welding_ellipses,
#     fps=25,
#     trail_length=15,  # показывать траектории за 15 кадров
#     show_trails=True
# )

In [44]:
# # Объединяем все датафреймы по frame_number
# df_combined = df_n_spatters.merge(
#     df_spatters_temperature, on='frame_number', how='outer'
# ).merge(
#     df_welding_temperature, on='frame_number', how='outer'
# ).merge(
#     df_stats, on='frame_number', how='outer'
# )

In [45]:
# df_combined

## Потоковая фильтрация термограмм

In [46]:
def create_filtered_videos_for_ids(thermogram_ids, 
                                   input_base_path='num_py_convert_new',
                                   output_base_path='num_py_convert_new/filtered',
                                   k_size=9, sigma=1.8,
                                   overwrite_all=False):
    """
    Создает фильтрованные версии видео для списка термограмм
    
    Args:
        thermogram_ids: список номеров термограмм
        input_base_path: путь к исходным файлам
        output_base_path: путь для сохранения фильтрованных файлов
        k_size: размер ядра для фильтра LoG
        sigma: параметр sigma для фильтра LoG
        overwrite_all: если True, перезаписывает все существующие файлы без запроса
    """
    
    processed = 0
    skipped = 0
    errors = []
    
    for thermogram_id in thermogram_ids:
        print(f"\n{'='*50}")
        print(f"Обработка термограммы {thermogram_id}")
        print(f"{'='*50}")
        
        try:
            # Формируем пути
            file_path = f'{input_base_path}/thermogram_{thermogram_id}.npy'
            output_path = f'{output_base_path}/thermogram_{thermogram_id}_filtered.npy'
            
            # Проверяем существование исходного файла
            if not os.path.exists(file_path):
                print(f"✗ Исходный файл не найден: {file_path}")
                errors.append(f"Термограмма {thermogram_id}: исходный файл не найден")
                continue
            
            # Проверяем существование выходного файла
            if os.path.exists(output_path) and not overwrite_all:
                # Если не overwrite_all, запрашиваем подтверждение
                answer = input(f"Файл {output_path} уже существует. Перезаписать? (y/n/a для всех): ").strip().lower()
                
                if answer == 'a':
                    overwrite_all = True
                    print("Будут перезаписаны все существующие файлы")
                elif answer != 'y':
                    print(f"✓ Пропускаем термограмму {thermogram_id} (файл уже существует)")
                    skipped += 1
                    continue
            
            print(f"✓ Загружаем: {file_path}")
            frames = np.load(file_path)
            print(f"  Размер исходных данных: {frames.shape}")
            
            # Обработка видео
            print(f"  Применяем фильтр LoG (k_size={k_size}, sigma={sigma})...")
            filtered_video = min_loc_LoG_video(frames, k_size=k_size, sigma=sigma)
            print(f"  Размер фильтрованных данных: {filtered_video.shape}")
            
            # Сохранение результата
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            np.save(output_path, filtered_video)
            
            processed += 1
            print(f"✓ Термограмма {thermogram_id} успешно обработана")
            print(f"  Сохранено в: {output_path}")
            
        except Exception as e:
            error_msg = f"Термограмма {thermogram_id}: {str(e)}"
            print(f"✗ Ошибка: {error_msg}")
            errors.append(error_msg)
            continue
    
    # Итоги обработки
    print(f"\n{'='*50}")
    print("ИТОГИ ОБРАБОТКИ")
    print(f"{'='*50}")
    print(f"Всего термограмм в списке: {len(thermogram_ids)}")
    print(f"Успешно обработано: {processed}")
    print(f"Пропущено (уже существуют): {skipped}")
    print(f"Ошибок: {len(errors)}")
    
    if errors:
        print("\nОшибки при обработке:")
        for error in errors:
            print(f"  - {error}")
    
    return processed, skipped, errors

## Функция потоковой обработки файлов

In [47]:
def process_multiple_thermograms_safe(thermogram_ids, 
                                    original_base_path='num_py_convert_new',
                                    filtered_base_path='num_py_convert_new/filtered'):
    """
    Безопасная обработка нескольких термограмм с разными путями
    
    Args:
        thermogram_ids: список номеров термограмм
        original_base_path: путь к оригинальным файлам (thermogram_X.npy)
        filtered_base_path: путь к фильтрованным файлам (thermogram_X_filtered.npy)
    """
    
    df_combined_all = pd.DataFrame()
    all_results = {}
    processed_count = 0
    
    for thermogram_id in thermogram_ids:
        print(f"\n=== Обрабатываем термограмму {thermogram_id} ===")
        
        try:
            # ПУТИ С УЧЕТОМ РАЗНЫХ СТРУКТУР
            file_path = f'{original_base_path}/thermogram_{thermogram_id}.npy'  # оригинал
            output_path = f'{filtered_base_path}/thermogram_{thermogram_id}_filtered.npy'  # фильтрованный
            
            import os
            if not os.path.exists(file_path):
                print(f"✗ Оригинальный файл не найден: {file_path}")
                continue
            if not os.path.exists(output_path):
                print(f"✗ Фильтрованный файл не найден: {output_path}")
                continue
            
            print(f"✓ Файлы найдены:")
            print(f"  Оригинал: {file_path}")
            print(f"  Фильтр: {output_path}")
            
            # Загрузка данных
            frames = np.load(file_path)
            filtered_video = np.load(output_path)
            print(f"✓ Загружено: {frames.shape} кадров")
            
            # Обработка
            spatters_list = detect_spatters_video(filtered_video)
            welding_boxes, welding_ellipses = detect_welding_zone_video(frames)
            
            cleaned_spatters = apply_dead_zone(spatters_list, welding_ellipses, dead_zone_radius=50)
            cleaned_spatters = apply_fixed_corridor_incremental(cleaned_spatters, welding_ellipses, corridor_width=35, lookahead=25)
            
            cleaned_spatters, hot_stripes_list = filter_spatters_by_hot_stripes_video_oriented(
                frames=frames,
                cleaned_spatters_list=cleaned_spatters,
                threshold_low=500,
                threshold_high=1200,  
                min_area=35,
                circular_thr=4.0,
                angle_smooth=0.3,
                initial_angle=-3*np.pi/4,
                show_progress=False
            )
            
            cleaned_spatters, static_spatters = remove_static_spatters(cleaned_spatters, max_shift=5, min_frames=10)
            
            # Расчет статистик
            df_n_spatters = create_spatters_count_dataframe(cleaned_spatters)
            df_spatters_temperature = calculate_spatters_median_temperature(
                cleaned_spatters=cleaned_spatters, frames=frames, threshold_temp=420
            )
            df_welding_temperature = calculate_welding_zone_temperature_threshold(
                welding_ellipses=welding_ellipses, frames=frames, threshold_temp=1000, radius=50, thermogram_id=thermogram_id
            )
            
            cleaned_spatters_id, df_stats = track_spatters_with_ids(
                cleaned_spatters=cleaned_spatters,
                frame_width=frames.shape[2],
                frame_height=frames.shape[1],
                min_movement_distance=5,
                min_movement_frames=3
            )
            
            # Объединение
            df_combined = df_n_spatters.merge(df_spatters_temperature, on='frame_number', how='outer')
            df_combined = df_combined.merge(df_welding_temperature, on='frame_number', how='outer')
            df_combined = df_combined.merge(df_stats, on='frame_number', how='outer')
            
            df_combined['thermogram_id'] = thermogram_id
            
            # Добавляем в общий датафрейм
            if df_combined_all.empty:
                df_combined_all = df_combined
            else:
                df_combined_all = pd.concat([df_combined_all, df_combined], ignore_index=True)
            
            all_results[thermogram_id] = {
                'df_combined': df_combined,
                'cleaned_spatters': cleaned_spatters,
                'cleaned_spatters_id': cleaned_spatters_id,
                'welding_ellipses': welding_ellipses,
                'frames': frames
            }
            
            processed_count += 1
            print(f"✓ Термограмма {thermogram_id} успешно обработана")
            
        except Exception as e:
            print(f"✗ Ошибка при обработке термограммы {thermogram_id}: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    # Финальная сортировка
    if not df_combined_all.empty:
        df_combined_all = df_combined_all.sort_values(['thermogram_id', 'frame_number']).reset_index(drop=True)
    
    print(f"\n=== ИТОГИ ===")
    print(f"Успешно обработано: {processed_count}/{len(thermogram_ids)} термограмм")
    if not df_combined_all.empty:
        print(f"Всего строк данных: {len(df_combined_all)}")
    
    return df_combined_all, all_results

## Потоковая обработка с цикличным сохранением

In [63]:
def process_multiple_thermograms_safe(thermogram_ids, 
                                    original_base_path='num_py_convert_new',
                                    filtered_base_path='num_py_convert_new/filtered',
                                    save_dir='processing_progress'):
    """
    Безопасная обработка нескольких термограмм с сохранением прогресса
    
    Args:
        thermogram_ids: список номеров термограмм для обработки
        original_base_path: путь к оригинальным файлам (thermogram_X.npy)
        filtered_base_path: путь к фильтрованным файлам (thermogram_X_filtered.npy)
        save_dir: директория для сохранения прогресса
    
    Returns:
        DataFrame с объединенными данными по всем обработанным термограммам
    """
    
    # Создаем директорию для сохранения прогресса, если ее нет
    os.makedirs(save_dir, exist_ok=True)
    
    # Пути к файлам прогресса
    df_save_path = f'{save_dir}/df_combined_all.pkl'
    ids_save_path = f'{save_dir}/processed_thermogram_ids.pkl'
    
    # 1. ЗАГРУЗКА СУЩЕСТВУЮЩЕГО ПРОГРЕССА
    processed_thermogram_ids = []
    df_combined_all = pd.DataFrame()
    
    # Загружаем список уже обработанных ID
    if os.path.exists(ids_save_path):
        try:
            with open(ids_save_path, 'rb') as f:
                processed_thermogram_ids = pickle.load(f)
            print(f"✓ Загружены обработанные термограммы: {len(processed_thermogram_ids)} шт")
        except Exception as e:
            print(f"✗ Ошибка загрузки списка ID: {e}")
    
    # Загружаем сохраненные данные
    if os.path.exists(df_save_path):
        try:
            df_combined_all = pd.read_pickle(df_save_path)
            print(f"✓ Загружены сохраненные данные: {len(df_combined_all)} строк")
            if 'thermogram_id' in df_combined_all.columns:
                print(f"✓ В данных присутствуют термограммы: {sorted(df_combined_all['thermogram_id'].unique())}")
        except Exception as e:
            print(f"✗ Ошибка загрузки данных: {e}")
    
    # 2. ОПРЕДЕЛЕНИЕ НЕОБРАБОТАННЫХ ТЕРМОГРАММ
    unprocessed_ids = [tid for tid in thermogram_ids if tid not in processed_thermogram_ids]
    
    print(f"\n{'='*50}")
    print("СТАТУС ОБРАБОТКИ")
    print(f"{'='*50}")
    print(f"Всего запрошено: {len(thermogram_ids)} термограмм")
    print(f"Уже обработано: {len(processed_thermogram_ids)}")
    print(f"Нужно обработать: {len(unprocessed_ids)}")
    
    if len(processed_thermogram_ids) > 0:
        print(f"Обработанные: {sorted(processed_thermogram_ids)}")
    
    if len(unprocessed_ids) == 0:
        print("✓ Все запрошенные термограммы уже обработаны!")
        return df_combined_all
    
    # 3. ОБРАБОТКА НЕОБРАБОТАННЫХ ТЕРМОГРАММ
    successful_count = 0
    failed_count = 0
    failed_ids = []
    
    for thermogram_id in unprocessed_ids:
        print(f"\n{'='*40}")
        print(f"Термограмма {thermogram_id}")
        print(f"{'='*40}")
        
        try:
            # Проверяем существование файлов
            original_file = f'{original_base_path}/thermogram_{thermogram_id}.npy'
            filtered_file = f'{filtered_base_path}/thermogram_{thermogram_id}_filtered.npy'
            
            if not os.path.exists(original_file):
                print(f"✗ Оригинальный файл не найден: {original_file}")
                failed_ids.append(thermogram_id)
                failed_count += 1
                continue
                
            if not os.path.exists(filtered_file):
                print(f"✗ Фильтрованный файл не найден: {filtered_file}")
                failed_ids.append(thermogram_id)
                failed_count += 1
                continue
            
            print(f"✓ Файлы найдены")
            print(f"  Оригинал: {original_file}")
            print(f"  Фильтр: {filtered_file}")
            
            # Загрузка данных
            frames = np.load(original_file)
            filtered_video = np.load(filtered_file)
            print(f"✓ Загружено: {frames.shape[0]} кадров, {frames.shape[1]}x{frames.shape[2]}")
            
            # ПАЙПЛАЙН ОБРАБОТКИ
            print("┌─ Запуск пайплайна обработки...")
            
            # 3.1 Детекция брызг и зоны сварки
            print("├─ Детекция брызг и зоны сварки...")
            spatters_list = detect_spatters_video(filtered_video)
            welding_boxes, welding_ellipses = detect_welding_zone_video(frames)
            
            # 3.2 Фильтрация брызг
            print("├─ Фильтрация брызг (мертвая зона)...")
            cleaned_spatters = apply_dead_zone(spatters_list, welding_ellipses, dead_zone_radius=50)
            
            print("├─ Фильтрация брызг (коридор)...")
            cleaned_spatters = apply_fixed_corridor_incremental(cleaned_spatters, welding_ellipses, corridor_width=35, lookahead=25)
            
            print("├─ Фильтрация горячими полосами...")
            cleaned_spatters, _ = filter_spatters_by_hot_stripes_video_oriented(
                frames=frames,
                cleaned_spatters_list=cleaned_spatters,
                threshold_low=500,
                threshold_high=1200,  
                min_area=35,
                circular_thr=4.0,
                angle_smooth=0.3,
                initial_angle=-3*np.pi/4,
                show_progress=False
            )
            
            print("├─ Удаление статичных брызг...")
            cleaned_spatters, _ = remove_static_spatters(cleaned_spatters, max_shift=5, min_frames=10)
            
            # 3.3 Расчет статистик
            print("├─ Расчет статистик...")
            df_n_spatters = create_spatters_count_dataframe(cleaned_spatters)
            df_spatters_temperature = calculate_spatters_median_temperature(
                cleaned_spatters=cleaned_spatters, 
                frames=frames, 
                threshold_temp=420
            )
            df_welding_temperature = calculate_welding_zone_temperature_threshold(
                welding_ellipses=welding_ellipses, 
                frames=frames, 
                threshold_temp=1000, 
                radius=50, 
                thermogram_id=thermogram_id
            )
            
            print("├─ Трекинг брызг...")
            cleaned_spatters_id, df_stats = track_spatters_with_ids(
                cleaned_spatters=cleaned_spatters,
                frame_width=frames.shape[2],
                frame_height=frames.shape[1],
                min_movement_distance=5,
                min_movement_frames=3
            )
            
            # 3.4 Объединение результатов
            print("├─ Объединение результатов...")
            df_combined = df_n_spatters.merge(df_spatters_temperature, on='frame_number', how='outer')
            df_combined = df_combined.merge(df_welding_temperature, on='frame_number', how='outer')
            df_combined = df_combined.merge(df_stats, on='frame_number', how='outer')
            
            df_combined['thermogram_id'] = thermogram_id
            
            # 3.5 Добавление к общим данным
            if df_combined_all.empty:
                df_combined_all = df_combined
            else:
                df_combined_all = pd.concat([df_combined_all, df_combined], ignore_index=True)
            
            # 4. СОХРАНЕНИЕ ПРОГРЕССА
            processed_thermogram_ids.append(thermogram_id)
            successful_count += 1
            
            # Сохраняем после каждой успешной термограммы
            df_combined_all.to_pickle(df_save_path)
            with open(ids_save_path, 'wb') as f:
                pickle.dump(processed_thermogram_ids, f)
            
            print(f"└─ ✓ Термограмма {thermogram_id} успешно обработана")
            print(f"   Всего обработано: {len(processed_thermogram_ids)}")
            print(f"   Всего строк данных: {len(df_combined_all)}")
            
        except Exception as e:
            print(f"└─ ✗ Ошибка при обработке термограммы {thermogram_id}: {e}")
            import traceback
            traceback.print_exc()
            failed_ids.append(thermogram_id)
            failed_count += 1
            continue
    
    # 5. ФИНАЛЬНАЯ ОБРАБОТКА
    if not df_combined_all.empty:
        df_combined_all = df_combined_all.sort_values(['thermogram_id', 'frame_number']).reset_index(drop=True)
        
        # Финальное сохранение
        df_combined_all.to_pickle(df_save_path)
        with open(ids_save_path, 'wb') as f:
            pickle.dump(processed_thermogram_ids, f)
    
    # 6. ИТОГИ
    print(f"\n{'='*50}")
    print("ИТОГИ ОБРАБОТКИ")
    print(f"{'='*50}")
    print(f"Запрошено термограмм: {len(thermogram_ids)}")
    print(f"Успешно обработано: {successful_count}")
    print(f"Ошибок/пропусков: {failed_count}")
    
    if failed_count > 0:
        print(f"Термограммы с ошибками: {sorted(failed_ids)}")
    
    if not df_combined_all.empty:
        print(f"\nРЕЗУЛЬТАТЫ:")
        print(f"  Всего строк данных: {len(df_combined_all)}")
        print(f"  Термограмм в данных: {df_combined_all['thermogram_id'].nunique()}")
        print(f"  Диапазон кадров: {df_combined_all['frame_number'].min()} - {df_combined_all['frame_number'].max()}")
        
        # Статистика по термограммам
        thermogram_stats = df_combined_all.groupby('thermogram_id').size()
        print(f"  Строк по термограммам:")
        for thermogram_id, count in thermogram_stats.items():
            print(f"    - Термограмма {thermogram_id}: {count} кадров")
    
    print(f"\nФайлы сохранены в '{save_dir}':")
    print(f"  • df_combined_all.pkl - данные ({len(df_combined_all)} строк)")
    print(f"  • processed_thermogram_ids.pkl - список ID ({len(processed_thermogram_ids)} ID)")
    
    return df_combined_all


def load_saved_processing_results(save_dir='processing_progress'):
    """
    Загрузить сохраненные результаты обработки
    
    Args:
        save_dir: директория с сохраненными результатами
    
    Returns:
        tuple: (DataFrame с данными, список обработанных ID)
        Если файлы не найдены, возвращает (None, [])
    """
    
    df_save_path = f'{save_dir}/df_combined_all.pkl'
    ids_save_path = f'{save_dir}/processed_thermogram_ids.pkl'
    
    if not os.path.exists(df_save_path):
        print("✗ Файл с данными не найден")
        return None, []
    
    try:
        # Загрузка данных
        df_combined_all = pd.read_pickle(df_save_path)
        
        # Загрузка списка ID
        processed_thermogram_ids = []
        if os.path.exists(ids_save_path):
            with open(ids_save_path, 'rb') as f:
                processed_thermogram_ids = pickle.load(f)
        
        print(f"✓ Результаты загружены:")
        print(f"  • Данные: {len(df_combined_all)} строк")
        print(f"  • Обработанные термограммы: {len(processed_thermogram_ids)} шт")
        
        if not df_combined_all.empty and 'thermogram_id' in df_combined_all.columns:
            unique_ids = sorted(df_combined_all['thermogram_id'].unique())
            print(f"  • Термограммы в данных: {unique_ids}")
        
        return df_combined_all, processed_thermogram_ids
        
    except Exception as e:
        print(f"✗ Ошибка при загрузке результатов: {e}")
        import traceback
        traceback.print_exc()
        return None, []

## Запуск потоковой фильтрации

In [49]:
thermogram_ids = [7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 
                  21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 
                  35, 36, 37, 38, 39, 40, 41, 42, 43]

# processed, skipped, errors = create_filtered_videos_for_ids(
#     thermogram_ids=thermogram_ids,
#     overwrite_all=False
# )

## Запуск сборки датафрейма

In [ ]:
# Список термограмм для обработки
# Теперь указываем оба пути
# thermogram_ids = [7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 
#                   21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 
#                   35, 36, 37, 38, 39, 40, 41, 42, 43]

thermogram_ids = [21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34]


df_combined_all = process_multiple_thermograms_safe(
    thermogram_ids=thermogram_ids,
    original_base_path='num_py_convert_new',           # путь к оригиналам
    filtered_base_path='num_py_convert_new/filtered' # путь к фильтрованным
)

✓ Загружены обработанные термограммы: 5 шт
✓ Загружены сохраненные данные: 7500 строк
✓ В данных присутствуют термограммы: [np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20)]

СТАТУС ОБРАБОТКИ
Всего запрошено: 13 термограмм
Уже обработано: 5
Нужно обработать: 8
Обработанные: [16, 17, 18, 19, 20]

Термограмма 7
✓ Файлы найдены
  Оригинал: num_py_convert_new/thermogram_7.npy
  Фильтр: num_py_convert_new/filtered/thermogram_7_filtered.npy
✓ Загружено: 1500 кадров, 256x320
┌─ Запуск пайплайна обработки...
├─ Детекция брызг и зоны сварки...


Детекция зон сварки с предсказанием: 100%|██████████| 1500/1500 [00:00<00:00, 3671.94it/s]


├─ Фильтрация брызг (мертвая зона)...
├─ Фильтрация брызг (коридор)...


Filtering spatters: 100%|██████████| 1500/1500 [02:17<00:00, 10.95it/s]


├─ Фильтрация горячими полосами...
├─ Удаление статичных брызг...
├─ Расчет статистик...
├─ Трекинг брызг...
├─ Объединение результатов...
└─ ✓ Термограмма 7 успешно обработана
   Всего обработано: 6
   Всего строк данных: 9000

Термограмма 8
✓ Файлы найдены
  Оригинал: num_py_convert_new/thermogram_8.npy
  Фильтр: num_py_convert_new/filtered/thermogram_8_filtered.npy
✓ Загружено: 1500 кадров, 256x320
┌─ Запуск пайплайна обработки...
├─ Детекция брызг и зоны сварки...


...

├─ Фильтрация горячими полосами...
├─ Удаление статичных брызг...
├─ Расчет статистик...
├─ Трекинг брызг...
├─ Объединение результатов...
└─ ✓ Термограмма 13 успешно обработана
   Всего обработано: 12
   Всего строк данных: 18000

Термограмма 14
✓ Файлы найдены
  Оригинал: num_py_convert_new/thermogram_14.npy
  Фильтр: num_py_convert_new/filtered/thermogram_14_filtered.npy
✓ Загружено: 1500 кадров, 256x320
┌─ Запуск пайплайна обработки...
├─ Детекция брызг и зоны сварки...


Детекция зон сварки с предсказанием: 100%|██████████| 1500/1500 [00:00<00:00, 3685.38it/s]


├─ Фильтрация брызг (мертвая зона)...
├─ Фильтрация брызг (коридор)...


Filtering spatters: 100%|██████████| 1500/1500 [02:12<00:00, 11.28it/s]


├─ Фильтрация горячими полосами...
├─ Удаление статичных брызг...
├─ Расчет статистик...
├─ Трекинг брызг...
├─ Объединение результатов...
└─ ✓ Термограмма 14 успешно обработана
   Всего обработано: 13
   Всего строк данных: 19500

ИТОГИ ОБРАБОТКИ
Запрошено термограмм: 13
Успешно обработано: 8
Ошибок/пропусков: 0

РЕЗУЛЬТАТЫ:
  Всего строк данных: 19500
  Термограмм в данных: 13
  Диапазон кадров: 1 - 1500
  Строк по термограммам:
    - Термограмма 7: 1500 кадров
    - Термограмма 8: 1500 кадров
    - Термограмма 9: 1500 кадров
    - Термограмма 10: 1500 кадров
    - Термограмма 11: 1500 кадров
    - Термограмма 12: 1500 кадров
    - Термограмма 13: 1500 кадров
    - Термограмма 14: 1500 кадров
    - Термограмма 16: 1500 кадров
    - Термограмма 17: 1500 кадров
    - Термограмма 18: 1500 кадров
    - Термограмма 19: 1500 кадров
    - Термограмма 20: 1500 кадров

Файлы сохранены в 'processing_progress':
  • df_combined_all.pkl - данные (19500 строк)
  • processed_thermogram_ids.pkl - сп

In [75]:
df_combined_all

,frame_number,spatters_count_del,thermogram_id,spatters_above_threshold,spatters_median_temperature,spatters_mean_temperature,spatters_max_temperature,spatters_total_pixels_above_threshold,spatters_percentage_above_threshold,welding_zone_detected,welding_zone_mean_temperature,welding_zone_max_temperature,welding_zone_size,test_zone,spatters_total,spatters_new,spatters_lost,spatters_mean_velocity,spatters_mean_area
0,1,0,7,0,NaN,NaN,NaN,0,0.0,False,NaN,NaN,0,None,0,0,0,0.0,0.0
1,2,0,7,0,NaN,NaN,NaN,0,0.0,False,NaN,NaN,0,None,0,0,0,0.0,0.0
2,3,0,7,0,NaN,NaN,NaN,0,0.0,False,NaN,NaN,0,None,0,0,0,0.0,0.0
3,4,0,7,0,NaN,NaN,NaN,0,0.0,False,NaN,NaN,0,None,0,0,0,0.0,0.0
4,5,0,7,0,NaN,NaN,NaN,0,0.0,False,NaN,NaN,0,None,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19495,1496,0,20,0,NaN,NaN,NaN,0,0.0,True,1130.245902,1386.0,61,D,0,0,0,0.0,0.0
19496,1497,0,20,0,NaN,NaN,NaN,0,0.0,True,1124.965517,1363.0,58,D,0,0,0,0.0,0.0
19497,1498,0,20,0,NaN,NaN,NaN,0,0.0,True,1121.129630,1340.0,54,D,0,0,0,0.0,0.0
19498,1499,0,20,0,NaN,NaN,NaN,0,0.0,True,1118.500000,1318.0,50,D,0,0,0,0.0,0.0


In [68]:
df_combined_all = load_saved_processing_results(save_dir='processing_progress')

✓ Результаты загружены:
  • Данные: 4500 строк
  • Обработанные термограммы: 3 шт
  • Термограммы в данных: [np.int64(16), np.int64(17), np.int64(18)]


In [ ]:
df_combined_all.to_csv('CSV/data_term_21_34.csv')

# Пояснение к столбцам датафрейма `df_combined_all`

## **Основная информация**
- **`frame_number`** - номер кадра в видео последовательности
- **`thermogram_id`** - идентификатор термограммы (номера 7, 8 и т.д.)

## **Статистика по брызгам (спаттерам)**
- **`spatters_count_del`** - количество обнаруженных брызг после применения всех фильтров (мертвая зона, коридор, горячие полосы)
- **`spatters_above_threshold`** - количество брызг с температурой выше порога (420°C)
- **`spatters_median_temperature`** - медианная температура всех брызг в кадре
- **`spatters_mean_temperature`** - средняя температура брызг в кадре
- **`spatters_max_temperature`** - максимальная температура среди всех брызг в кадре
- **`spatters_total_pixels_above_threshold`** - общее количество пикселей брызг с температурой выше порога
- **`spatters_percentage_above_threshold`** - процент брызг с температурой выше порога от общего количества

## **Зона сварки (welding zone)**
- **`welding_zone_detected`** - флаг обнаружения зоны сварки (True/False)
- **`welding_zone_mean_temperature`** - средняя температура в зоне сварки (пиксели выше 1000°C)
- **`welding_zone_max_temperature`** - максимальная температура в зоне сварки
- **`welding_zone_size`** - размер сварочной ванны (количество пикселей выше порога температуры)
- **`test_zone`** - фаза сварки (A, B, C, D) - делит период активной сварки на 4 равные части

## **Трекинг брызг (динамика)**
- **`spatters_total`** - общее количество активных (движущихся) брызг с назначенными ID
- **`spatters_new`** - количество новых брызг, появившихся в текущем кадре
- **`spatters_lost`** - количество брызг, которые пропали/исчезли в текущем кадре
- **`spatters_mean_velocity`** - средняя скорость движения брызг (пикселей/кадр)
- **`spatters_mean_area`** - средняя площадь брызг (пикселей²)

---

## **Ключевые особенности анализа:**

### **Температурные пороги:**
- **Брызги**: 420°C
- **Зона сварки**: 1000°C

### **Фильтрация:**
- Удалены статичные брызги (движение <5px за 3 кадра)
- Применена мертвая зона вокруг сварочной ванны (50px)
- Использован коридор для фильтрации траекторий
- Учтены горячие полосы для дополнительной фильтрации

### **Фазы сварки (test_zone):**
- **A** - первая четверть активной сварки
- **B** - вторая четверть  
- **C** - третья четверть
- **D** - четвертая четверть
- **NaN** - неактивная фаза сварки